In [1]:
import os
import re
import time
import pickle
import asyncio
import nest_asyncio
import pandas as pd
import numpy as np
import json
import logging
from tqdm import tqdm
from collections import defaultdict
from IPython.display import clear_output
from typing import List, Dict, Any, Optional
from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_community.chat_models import ChatClovaX
from pymilvus import connections, utility
from langchain_community.vectorstores.milvus import Milvus
from langchain.schema import BaseRetriever, Document
from sklearn.metrics.pairwise import cosine_similarity
from pydantic import (
    BaseModel,
    Field,
)  # BaseModel/Field 는 현재 코드에서 직접 사용되지 않지만, Langchain 내부 의존성 가능성 있음

from langchain_elasticsearch import ElasticsearchRetriever
from elasticsearch import Elasticsearch, helpers
from langchain_elasticsearch import ElasticsearchStore
from langchain.retrievers import EnsembleRetriever

In [2]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

##### Custom Retriever

In [ ]:
logger = logging.getLogger(__name__)


class ISBNMergingRetriever(BaseRetriever):
    """
    검색된 Document 리스트를 ISBN 기준으로 그룹화하고,
    같은 ISBN을 가진 문서들의 page_content를 병합하며,
    그룹 내에서 가장 정보가 많은 메타데이터를 대표로 사용합니다.
    """

    base_retriever: BaseRetriever

    def _extract_isbn(self, doc: Document) -> Optional[str]:
        """
        다양한 중첩 구조에서 ISBN 값을 추출합니다. (기존 로직 유지)
        """
        try:
            # Case 1: 평평한 구조
            for key in ["ISBN", "isbn"]:
                if key in doc.metadata:
                    isbn_val = doc.metadata[key]
                    # .0 제거 및 문자열 변환 추가
                    return str(isbn_val).replace(".0", "").strip() if isbn_val else None

            # Case 2: 1단계 중첩 ('metadata' 키 안에 메타데이터가 있는 경우 - ES Retriever 수정 후 이 경로 예상)
            inner_meta = doc.metadata.get("metadata", {})
            if isinstance(inner_meta, dict):
                for key in ["ISBN", "isbn"]:
                    if key in inner_meta:
                        isbn_val = inner_meta[key]
                        return (
                            str(isbn_val).replace(".0", "").strip()
                            if isbn_val
                            else None
                        )

                # Case 3: 2단계 중첩 (혹시 모를 경우 대비)
                inner_inner_meta = inner_meta.get("metadata", {})
                if isinstance(inner_inner_meta, dict):
                    for key in ["ISBN", "isbn"]:
                        if key in inner_inner_meta:
                            isbn_val = inner_inner_meta[key]
                            return (
                                str(isbn_val).replace(".0", "").strip()
                                if isbn_val
                                else None
                            )

            # Case 4: Elasticsearch `_source.metadata.ISBN` 구조 (ES Retriever 수정 전 대비)
            source_meta = doc.metadata.get("_source", {}).get("metadata", {})
            if isinstance(source_meta, dict):
                for key in ["ISBN", "isbn"]:
                    if key in source_meta:
                        isbn_val = source_meta[key]
                        return (
                            str(isbn_val).replace(".0", "").strip()
                            if isbn_val
                            else None
                        )

        except Exception as e:
            logger.warning(
                f"[ISBN 추출 오류] Metadata: {doc.metadata}, Error: {e}"
            )  # 오류 시 메타데이터 로깅

        logger.debug(
            f"ISBN 추출 실패 - Metadata: {doc.metadata}"
        )  # 실패 시 메타데이터 로깅
        return None

    def _get_best_metadata(self, doc_list: List[Document]) -> Dict[str, Any]:
        """
        주어진 문서 리스트(같은 ISBN 그룹) 내에서 가장 정보가 많은 메타데이터를 선택합니다.
        정보량 기준: 필수 키('title', 'author', 'ISBN') 존재 여부 및 전체 키 개수
        """
        best_meta = {}
        max_score = -1

        required_keys = {"title", "author", "ISBN"}  # 필수 메타데이터 키 정의

        for doc in doc_list:
            current_meta = doc.metadata
            score = 0

            # 1. 필수 키 존재 점수
            #    (소문자로 변환하여 키 비교 - 대소문자 문제 방지)
            keys_lower = {k.lower() for k in current_meta.keys()}
            required_keys_lower = {rk.lower() for rk in required_keys}
            score += sum(
                1
                for req_key in required_keys_lower
                if req_key in keys_lower
                and current_meta.get(
                    next((k for k in current_meta if k.lower() == req_key), None)
                )
            )  # 값이 있는 경우만 점수 부여

            # 2. 전체 키 개수 점수 (가중치 낮게)
            score += len(current_meta) * 0.1

            # 최고 점수 메타데이터 갱신
            if score > max_score:
                max_score = score
                best_meta = current_meta

        # 최종적으로 선택된 메타데이터 로깅 (디버깅용)
        if best_meta:
            logger.debug(
                f"선택된 최적 메타데이터 (Score: {max_score:.2f}): {best_meta}"
            )
        else:
            # 그룹 내 모든 문서의 메타데이터가 비어있는 극단적인 경우
            logger.warning(
                f"ISBN 그룹 내에서 유효한 메타데이터를 찾지 못함. 첫 번째 문서 메타데이터 사용 시도."
            )
            if doc_list:  # 문서 목록이 비어있지 않다면
                best_meta = doc_list[
                    0
                ].metadata  # 최소한 첫 번째 문서 메타데이터라도 반환

        # 선택된 메타데이터 반환 (복사본)
        return dict(best_meta)

    def _merge_documents_by_isbn(
        self, docs: List[Document], is_async=False
    ) -> List[Document]:
        grouped = defaultdict(list)
        merged_docs = []

        logger.info(
            f"[{'Async' if is_async else 'Sync'}] 병합 전 입력 문서 수: {len(docs)}"
        )

        # 1. ISBN 기준으로 그룹화
        for idx, doc in enumerate(docs):
            isbn = self._extract_isbn(doc)  # 수정된 ISBN 추출 로직 사용
            if isbn:
                grouped[isbn].append(doc)
            else:
                # ISBN 추출 실패 시 경고 로그 강화
                logger.warning(
                    f"[{'Async' if is_async else 'Sync'}] Doc {idx} ISBN 추출 실패 - Metadata: {doc.metadata}"
                )

        # 2. 그룹별 병합 처리
        for isbn, doc_list in grouped.items():
            # --- <<< 메타데이터 선택 로직 변경 >>> ---
            # 그룹 내 문서 리스트에서 가장 정보가 많은 메타데이터를 선택
            merged_meta = self._get_best_metadata(doc_list)
            # --- <<< 변경 끝 >>> ---

            # 페이지 내용 병합 (기존 로직 유지)
            combined_text = "\n\n---\n\n".join(
                d.page_content for d in doc_list if d.page_content
            ).strip()

            # 병합된 내용과 선택된 메타데이터로 새 Document 생성
            if combined_text:
                # 병합된 문서 생성 전, 최종 메타데이터 확인 (디버깅)
                logger.debug(f"ISBN {isbn} 병합: 최종 사용될 메타데이터: {merged_meta}")
                merged_docs.append(
                    Document(page_content=combined_text, metadata=merged_meta)
                )
            else:
                logger.warning(
                    f"[{'Async' if is_async else 'Sync'}] ISBN {isbn} 병합 후 내용 없음. 제외됨. 사용된 메타데이터: {merged_meta}"
                )

        logger.info(
            f"[{'Async' if is_async else 'Sync'}] ISBN 병합 그룹 수: {len(grouped)}"
        )
        logger.info(
            f"[{'Async' if is_async else 'Sync'}] 병합 후 최종 문서 수: {len(merged_docs)}"
        )
        return merged_docs

    # _get_relevant_documents, _aget_relevant_documents 메서드는 변경 없음
    def _get_relevant_documents(self, query: str) -> List[Document]:
        try:
            docs = self.base_retriever.get_relevant_documents(query)
        except Exception as e:
            logger.error(
                f"기본 리트리버 동기 검색 오류: '{query}' → {e}", exc_info=True
            )
            docs = []
        logger.info(f"Sync 검색 결과 총 문서 수: {len(docs)}")
        return self._merge_documents_by_isbn(docs, is_async=False)

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        try:
            docs = await self.base_retriever.aget_relevant_documents(query)
        except Exception as e:
            logger.error(
                f"기본 리트리버 비동기 검색 오류: '{query}' → {e}", exc_info=True
            )
            docs = []
        logger.info(f"Async 검색 결과 총 문서 수: {len(docs)}")
        return self._merge_documents_by_isbn(docs, is_async=True)

In [4]:
# text는 아래와 같은 형태로 들어가서 청크

# 제목 : [값]
# 분류 : [값]
# 저자 : [값]
# 저자소개 : [값]
# 책 소개 : [값]
# 목차 : [값]
# 출판사리뷰 : [값]

# 임베딩 pkl에 포함된 메타데이터 컬럼과 (임베딩,원본text)로 묶인 컬럼

# 메타데이터 및 벡터 문서 컬럼 설정

# metadata_columns = [
#     "ISBN",
#     "페이지",
#     "가격",
#     "제목",
#     "부제",
#     "저자",
#     "분류",
#     "목차",
#     "발행자",
#     "표지",
# ]
# vector_doc_columns = [
#     "제목",
#     "부제",
#     "분류",
#     "저자",
#     "저자소개",
#     "책소개",
#     "출판사리뷰",
#     "추천사",
#     "목차",
# ]

##### Util Function

In [30]:
def is_similar_question(new_emb, prev_embeds, threshold=0.65):
    """새 질문 임베딩이 이전 질문 임베딩 목록과 유사한지 확인"""
    if not prev_embeds:
        return False
    # 코사인 유사도 계산
    sim_scores = cosine_similarity([new_emb], prev_embeds)[0]
    max_score = np.max(sim_scores)  # 가장 높은 유사도 점수
    logger.info(
        f"[질문 유사도 판단] Max Similarity = {max_score:.3f} (Threshold = {threshold})"
    )
    return max_score > threshold


def extract_field(text, field_name):
    """주어진 텍스트에서 '필드명 : 값' 형식의 라인을 찾아 값을 추출"""
    # 정규 표현식 패턴: 시작 부분 공백 허용, 필드명, 콜론(:), 값, 끝 부분 공백 허용
    pattern = rf"^\s*{re.escape(field_name)}\s*[:：]\s*(.*?)\s*$"
    lines = text.splitlines()
    for line in lines:
        match = re.search(pattern, line, re.IGNORECASE)  # 대소문자 구분 없이 매칭
        if match:
            return match.group(1).strip()  # 매칭된 값의 양쪽 공백 제거 후 반환
    return ""  # 매칭되는 필드가 없으면 빈 문자열 반환

##### 환경설정 & 임베딩

In [31]:
load_dotenv()

api_key = os.getenv("NCP_CLOVASTUDIO_API_KEY")
api_url = os.getenv("NCP_CLOVASTUDIO_API_URL", "https://clovastudio.stream.ntruss.com/")
milvus_host = os.getenv("MILVUS_HOST", "localhost")
milvus_port = os.getenv("MILVUS_PORT", "19530")
es_url = os.getenv("ELASTICSEARCH_URL", "http://localhost:9200")
es_index_name = "book_bm25_index_v2"  # 인덱스 이름 변경 (SelfQuery 제거 반영)

# 필수 환경 변수 확인
if not api_key:
    raise ValueError("NCP_CLOVASTUDIO_API_KEY 환경 변수가 설정되지 않았습니다.")
if not api_url:
    raise ValueError("NCP_CLOVASTUDIO_API_URL 환경 변수가 설정되지 않았습니다.")
if not es_url:
    raise ValueError("ELASTICSEARCH_URL 환경 변수가 설정되지 않았습니다.")

# Langchain이 사용할 환경 변수 설정 (필요시)
os.environ["NCP_CLOVASTUDIO_API_KEY"] = api_key
os.environ["NCP_CLOVASTUDIO_API_URL"] = api_url

# --- 모델 초기화 ---
try:
    ncp_embeddings = ClovaXEmbeddings(model="bge-m3")  # ClovaX 임베딩 모델
    llm_clova = ChatClovaX(model="HCX-003", max_tokens=2048)  # ClovaX 챗 모델
    logger.info("ClovaX 임베딩 및 Chat 모델 초기화 완료")
except Exception as e:
    logger.error(f"ClovaX 모델 초기화 실패: {e}")
    raise

2025-04-07 14:58:17,406 - INFO - ClovaX 임베딩 및 Chat 모델 초기화 완료


In [32]:
embedding_file = r"C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\final_embedding\final_embedding.pkl"
if os.path.exists(embedding_file):
    try:
        with open(embedding_file, "rb") as f:
            saved_data = pickle.load(f)
        # pkl 파일에서 (텍스트, 임베딩) 쌍과 메타데이터 리스트 추출
        all_text_embedding_pairs = [
            (v["text"], v["embedding"])
            for v in saved_data.values()
            if "text" in v and "embedding" in v
        ]
        all_metadata_list = [
            v["metadata"] for v in saved_data.values() if "metadata" in v
        ]
        # 로드된 데이터 개수 일치 확인
        if len(all_text_embedding_pairs) != len(all_metadata_list):
            logger.warning(
                f"로드된 텍스트/임베딩 쌍({len(all_text_embedding_pairs)})과 메타데이터({len(all_metadata_list)}) 개수 불일치."
            )
            min_len = min(len(all_text_embedding_pairs), len(all_metadata_list))
            all_text_embedding_pairs = all_text_embedding_pairs[:min_len]
            all_metadata_list = all_metadata_list[:min_len]
            logger.info(f"데이터를 {min_len}개로 조정하여 계속 진행.")

        logger.info(f"임베딩 데이터 로드 완료: {len(all_text_embedding_pairs)}개")
    except Exception as e:
        logger.error(f"임베딩 파일 로드 실패: {e}", exc_info=True)
        raise
else:
    raise FileNotFoundError(f"임베딩 파일을 찾을 수 없음: {embedding_file}")

# --- 메타데이터 처리 ---
# 원본 키 -> 목표 키 매핑 정의 (Langchain Document의 metadata 필드명 기준)
metadata_mapping = {
    "ISBN": "ISBN",  # ISBN은 그대로 유지
    "페이지": "page",  # 페이지 수
    "가격": "price",  # 가격
    "제목": "title",  # 제목
    "부제": "subtitle",  # 부제
    "저자": "author",  # 저자
    "분류": "category",  # 분류/카테고리
    "저자소개": "author_intro",  # 저자 소개
    "책소개": "book_intro",  # 책 소개
    "목차": "table_of_contents",  # 목차
    "출판사리뷰": "publisher_review",  # 출판사 리뷰
    "추천사": "recommendation",  # 추천사
    "발행자": "publisher",  # 출판사/발행자
    "표지": "book_cover",  # 표지 URL (사용한다면)
}


# 메타데이터 클리닝 함수: NA값 처리, 타입 변환 등
def clean_metadata(meta: dict) -> dict:
    cleaned = {}
    target_keys = list(metadata_mapping.values())  # 목표 키 목록

    # 원본 키와 목표 키 매핑 역전
    original_to_target = {
        k_orig: k_target for k_orig, k_target in metadata_mapping.items()
    }

    for target_key in target_keys:
        # 해당 목표 키에 매핑된 원본 키 찾기
        original_key = next(
            (k for k, v in original_to_target.items() if v == target_key), None
        )

        # 원본 딕셔너리에서 값 가져오기 (원본 키 또는 목표 키 이름으로 시도)
        value = (
            meta.get(original_key)
            if original_key and original_key in meta
            else meta.get(target_key)
        )

        # 키별 타입 처리 및 NA 처리
        if pd.isna(value):
            if target_key in ["page", "price"]:
                cleaned[target_key] = 0
            else:
                cleaned[target_key] = ""
        elif target_key == "ISBN":  # ISBN 타입 에러 '.0'으로 끝나는 경우 제거
            try:
                str_value = str(value).strip()
                if str_value.endswith(".0"):
                    cleaned[target_key] = str_value[:-2]
                else:
                    cleaned[target_key] = str_value
            except Exception as e:
                logger.warning(
                    f"ISBN 값 '{value}' 처리 중 오류 발생 (기본 문자열 변환 사용): {e}"
                )
                cleaned[target_key] = str(value).strip()
        elif target_key == "subtitle":
            cleaned[target_key] = str(value)
        elif target_key in ["page", "price"]:
            try:
                cleaned[target_key] = int(float(value))
            except (ValueError, TypeError):
                logger.warning(
                    f"'{target_key}' 값 '{value}' 정수 변환 실패. 0으로 설정."
                )
                cleaned[target_key] = 0
        else:
            cleaned[target_key] = str(value)

    return cleaned


# --- Langchain Document 객체 생성 ---
documents = []
for i, (pair, meta) in enumerate(zip(all_text_embedding_pairs, all_metadata_list)):
    try:
        cleaned_meta = clean_metadata(meta)  # 메타데이터 클리닝
        # Document 객체 생성: page_content는 텍스트, metadata는 클리닝된 딕셔너리
        documents.append(Document(page_content=pair[0], metadata=cleaned_meta))
    except Exception as e:
        logger.error(
            f"{i}번째 데이터 처리 중 오류 발생: {e}. 메타데이터: {meta}", exc_info=True
        )
        # 오류 발생 시 해당 데이터 건너뛰거나 기본값으로 처리 가능

logger.info(f"총 {len(documents)}개의 Document 객체 생성 완료.")

# Document 리스트에서 텍스트, 임베딩, 메타데이터 분리 (이후 사용 위함)
texts = [doc.page_content for doc in documents]
embeds = [
    pair[1] for pair in all_text_embedding_pairs[: len(documents)]
]  # documents 개수에 맞춰 슬라이싱
metadatas = [doc.metadata for doc in documents]

2025-04-07 15:00:10,914 - INFO - 임베딩 데이터 로드 완료: 116218개
2025-04-07 15:01:22,576 - INFO - 총 116218개의 Document 객체 생성 완료.


In [ ]:
# --- Milvus 벡터 저장소 설정 ---
collection_name = "book_rag_db_v_no_sq"  # 컬렉션 name
temp_conn_alias = "utility_check_conn"  # 임시 연결

# 기존 컬렉션 확인 및 삭제 (임시 연결 사용)
try:
    connections.connect(alias=temp_conn_alias, host=milvus_host, port=milvus_port)
    logger.info(
        f"Milvus 유틸리티 함수용 임시 연결 설정 완료 (alias: {temp_conn_alias})."
    )
    if utility.has_collection(collection_name, using=temp_conn_alias):
        logger.warning(f"기존 Milvus 컬렉션 '{collection_name}'을 삭제.")
        utility.drop_collection(collection_name, using=temp_conn_alias)
    else:
        logger.info(
            f"Milvus 컬렉션 '{collection_name}'이(가) 존재하지 않음. 새로 생성."
        )
except Exception as e:
    logger.error(f"Milvus 유틸리티 함수 실행 중 오류 발생: {e}", exc_info=True)
finally:
    # 임시 연결 해제
    try:
        if connections.get_connection_addr(temp_conn_alias):
            connections.disconnect(temp_conn_alias)
            logger.info(
                f"Milvus 유틸리티 함수용 임시 연결 해제 완료 (alias: {temp_conn_alias})."
            )
    except Exception as disconnect_e:
        logger.warning(f"Milvus 임시 연결 해제 중 오류 (무시 가능): {disconnect_e}")

# Milvus 벡터 저장소 인스턴스 생성 및 데이터 추가
try:
    vectorstore = Milvus(
        embedding_function=ncp_embeddings,  # 사용할 임베딩 함수
        collection_name=collection_name,  # 사용할 컬렉션 이름
        connection_args={"host": milvus_host, "port": milvus_port},  # Milvus 연결 정보
        auto_id=True,  # Milvus가 자동으로 ID 생성하도록 설정
    )
    logger.info(
        f"Langchain Milvus 인스턴스 초기화 및 연결 설정 완료 (컬렉션: '{collection_name}')."
    )

    # --- 사전 계산된 임베딩 사용 로직 (Monkey Patching) ---
    # 기존 embed_documents 메소드 백업
    original_embed_documents = ClovaXEmbeddings.embed_documents

    # 사전 계산된 임베딩을 반환하는 새 메소드 정의
    def precomputed_embed_documents(cls, input_texts: List[str]) -> List[List[float]]:
        # add_texts 호출 시 전달된 texts 리스트와 미리 로드한 texts 리스트가 정확히 일치하는 경우,
        # 미리 로드한 embeds 리스트를 반환 (API 호출 회피)
        # 주의: 이 방식은 add_texts 호출 시점에 texts 순서와 내용이 정확히 일치해야 함
        if len(input_texts) == len(texts) and all(
            t1 == t2 for t1, t2 in zip(input_texts, texts)
        ):
            logger.info(f"사전 계산된 임베딩 사용: {len(embeds)}개")
            return embeds
        else:
            # 일치하지 않으면 원래 embed_documents 메소드 호출
            logger.warning(
                "입력 텍스트가 사전 계산된 데이터와 불일치. ClovaX API를 통해 임베딩 수행."
            )
            # 원래 클래스 메소드를 호출하는 방식으로 변경
            return original_embed_documents.__func__(cls, input_texts)

    # ClovaXEmbeddings 클래스의 embed_documents 메소드를 새 메소드로 교체 (Patch)
    ClovaXEmbeddings.embed_documents = classmethod(precomputed_embed_documents)

    # 데이터 추가 (이 때 precomputed_embed_documents가 호출됨)
    vectorstore.add_texts(texts=texts, metadatas=metadatas)
    logger.info(f"Milvus에 {len(texts)}개의 텍스트와 임베딩 추가 완료.")

    # 작업 완료 후 원래 메소드로 복구
    ClovaXEmbeddings.embed_documents = original_embed_documents
    logger.info("ClovaXEmbeddings.embed_documents 메소드 원상 복구 완료.")

except Exception as e:
    logger.error(f"Milvus 데이터 추가 중 오류 발생: {e}", exc_info=True)
    # 오류 발생 시에도 반드시 원래 메소드로 복구 시도
    if (
        "original_embed_documents" in locals()
        and hasattr(ClovaXEmbeddings, "embed_documents")
        and ClovaXEmbeddings.embed_documents != original_embed_documents
    ):
        ClovaXEmbeddings.embed_documents = original_embed_documents
        logger.info(
            "오류 발생 후 ClovaXEmbeddings.embed_documents 메소드 원상 복구 시도 완료."
        )
    raise

2025-04-07 15:01:28,278 - INFO - Milvus 유틸리티 함수용 임시 연결 설정 완료 (alias: utility_check_conn).
2025-04-07 15:01:28,294 - WARNING - 기존 Milvus 컬렉션 'book_rag_db_v_no_sq'을 삭제.
2025-04-07 15:01:28,454 - INFO - Milvus 유틸리티 함수용 임시 연결 해제 완료 (alias: utility_check_conn).
2025-04-07 15:01:28,463 - INFO - Langchain Milvus 인스턴스 초기화 및 연결 설정 완료 (컬렉션: 'book_rag_db_v_no_sq').
2025-04-07 15:01:28,509 - INFO - 사전 계산된 임베딩 사용: 116218개
2025-04-07 15:04:55,277 - INFO - Milvus에 116218개의 텍스트와 임베딩 추가 완료.
2025-04-07 15:04:55,903 - INFO - ClovaXEmbeddings.embed_documents 메소드 원상 복구 완료.


In [ ]:
# Elasticsearch store 설정
try:
    logger.info(f"Elasticsearch 연결 시도 중: {es_url}...")
    es_client = Elasticsearch(
        hosts=[es_url],
        request_timeout=120,
    )

    if not es_client.ping():
        raise ConnectionError("Elasticsearch 연결 실패. 서버 상태 및 URL 확인 필요.")
    logger.info("Elasticsearch 연결 성공")

    if es_client.indices.exists(index=es_index_name):
        logger.warning(f"기존 Elasticsearch 인덱스 '{es_index_name}' 삭제 중...")
        es_client.indices.delete(index=es_index_name, ignore=[400, 404])
        logger.info(f"기존 Elasticsearch 인덱스 '{es_index_name}' 삭제 완료.")
    else:
        logger.info(
            f"Elasticsearch 인덱스 '{es_index_name}' 존재하지 않음. 새로 생성됩니다."
        )

    # ElasticsearchStore 초기화 (BM25)
    es_store = ElasticsearchStore(
        index_name=es_index_name,
        es_connection=es_client,
        strategy=ElasticsearchStore.ExactRetrievalStrategy(),  # <-- 추가: BM25 검색 명시!!!
    )
    logger.info(
        f"Langchain ElasticsearchStore 인스턴스 초기화 완료 (인덱스: '{es_index_name}', BM25 검색 전용 - ExactRetrievalStrategy)."
    )

    # Elasticsearch 인덱싱 (Bulk API 사용)
    logger.info(f"Elasticsearch에 문서 인덱싱 시작 (총 {len(texts)}개)...")
    actions = [
        {
            "_index": es_index_name,
            "_source": {"text": texts[i], "metadata": metadatas[i]},
        }
        for i in range(len(texts))
    ]
    indexed_count, errors = helpers.bulk(es_client, actions, raise_on_error=False)
    logger.info(f"Elasticsearch 벌크 인덱싱 시도 완료: {indexed_count} 문서 성공.")
    if errors:
        logger.error(
            f"Elasticsearch 벌크 인덱싱 중 오류 발생: {len(errors)}개 문서 실패."
        )
        for i, error in enumerate(errors[:5]):
            logger.error(f"  실패 {i+1}: {error}")

    es_client.indices.refresh(index=es_index_name)
    logger.info(f"Elasticsearch 인덱스 '{es_index_name}' 새로고침 완료.")

except ConnectionError as ce:
    logger.error(f"Elasticsearch 연결 실패: {ce}")
    raise
except Exception as e:
    logger.error(
        f"Elasticsearch 설정 또는 데이터 추가 중 오류 발생: {e}", exc_info=True
    )
    raise

2025-04-07 15:16:29,847 - INFO - Elasticsearch 연결 시도 중: http://localhost:9200...
2025-04-07 15:16:33,478 - INFO - HEAD http://localhost:9200/ [status:200 duration:3.619s]
2025-04-07 15:16:33,479 - INFO - Elasticsearch 연결 성공
2025-04-07 15:16:33,793 - INFO - HEAD http://localhost:9200/book_bm25_index_v2 [status:200 duration:0.314s]
2025-04-07 15:16:33,794 - WARNING - 기존 Elasticsearch 인덱스 'book_bm25_index_v2' 삭제 중...
C:\Users\computer\AppData\Local\Temp\ipykernel_7132\495359522.py:15: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.delete(index=es_index_name, ignore=[400, 404])
2025-04-07 15:16:48,414 - INFO - DELETE http://localhost:9200/book_bm25_index_v2 [status:200 duration:13.838s]
2025-04-07 15:16:48,416 - INFO - 기존 Elasticsearch 인덱스 'book_bm25_index_v2' 삭제 완료.
2025-04-07 15:16:48,417 - INFO - Langchain ElasticsearchStore 인스턴스 초기화 완료 (인덱스: 'book_bm25_index_v2', BM25 검색 전용 - ExactRetrievalStrate

#####  Retriever 및 QA 체인

In [ ]:
# 1. Dense Retriever (Milvus)
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
logger.info("Dense retriever (Milvus 벡터 검색) 설정 완료 (k=5).")

# 2. Sparse Retriever (Elasticsearch - BM25)
try:
    if "es_client" not in locals() or not es_client.ping():
        raise ConnectionError(
            "Elasticsearch client가 준비되지 않았거나 연결할 수 없습니다."
        )

    sparse_retriever = ElasticsearchRetriever(
        es_client=es_client,
        index_name=es_index_name,
        body_func=lambda query: {
            "size": 5,
            "query": {"match": {"text": {"query": query}}},
        },  # ES에서 가져올 문서 수를 제한 (Ensemble이 최종 개수 조절)
        content_field="text",  # ES 문서의 text 필드를 Document.page_content로 사용
        metadata_field="metadata",
    )
    # 로그 메시지 업데이트
    logger.info(
        "Sparse retriever (Elasticsearch BM25) 설정 완료 (ElasticsearchRetriever 직접 사용, metadata_field 지정)."
    )

except ConnectionError as ce:
    logger.error(f"Elasticsearch 클라이언트 오류: {ce}")
    raise
except Exception as e:
    logger.error(f"ElasticsearchRetriever 직접 설정 중 오류 발생: {e}", exc_info=True)
    raise

# 3. Hybrid Retriever - Ensemble (Dense + Sparse)
hybrid_retriever = EnsembleRetriever(
    retrievers=[sparse_retriever, dense_retriever], weights=[0.5, 0.5], c=60
)
logger.info(
    "Hybrid ensemble retriever 설정 완료 (Direct ES BM25: 0.5, Milvus Vector: 0.5)."
)

# 4. Final Retriever with ISBN Merging
merged_hybrid_retriever = ISBNMergingRetriever(base_retriever=hybrid_retriever)
logger.info("ISBN Merging Retriever 설정 완료 (Hybrid 결과 기반).")

# 5. RetrievalQA Chain 설정 (최종 리트리버 사용)
dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_clova,
    retriever=merged_hybrid_retriever,
    return_source_documents=True,
)
logger.info("RetrievalQA 체인 설정 완료 (Merged Hybrid Retriever 사용).")

MIN_INFO_LENGTH = 10  # 요약 생성을 위한 최소 텍스트 길이
previous_additional_question_embeddings = (
    []
)  # 이전 추가 질문 임베딩 저장 리스트 (유사 질문 반복 방지 &~)

2025-04-07 15:21:14,998 - INFO - Dense retriever (Milvus 벡터 검색) 설정 완료 (k=5).
2025-04-07 15:21:15,242 - INFO - HEAD http://localhost:9200/ [status:200 duration:0.238s]
2025-04-07 15:21:15,365 - INFO - Sparse retriever (Elasticsearch BM25) 설정 완료 (ElasticsearchRetriever 직접 사용, metadata_field 지정).
2025-04-07 15:21:15,391 - INFO - Hybrid ensemble retriever 설정 완료 (Direct ES BM25: 0.5, Milvus Vector: 0.5).
2025-04-07 15:21:15,400 - INFO - ISBN Merging Retriever 설정 완료 (Hybrid 결과 기반).
2025-04-07 15:21:15,592 - INFO - RetrievalQA 체인 설정 완료 (Merged Hybrid Retriever 사용).


##### Prompt

In [143]:
# 1. 사용자 선호도 추출 프롬프트
extract_pref_prompt_v2 = PromptTemplate(
    input_variables=["text"],
    template="""
다음 사용자 발화에서 사용자의 선호도 및 책에 대한 요구사항을 아래 JSON 형식으로 추출해라. 각 항목은 관련된 정보가 명확할때만 명확한 항목에 포함(소설 -> category)하고, 없다면 빈 리스트 `[]` 또는 빈 문자열 `""`로 남겨라. 여러개가 추출될 수 있는 항목은 리스트로 추출하라.
사용자 입력에서 모호한 정보는 implicit info로 포함해라.
존재하지 않는 사용자 선호도 정보는 임의로 생성하지 마라.

입력: {{ text }}

출력 형식 (JSON, 다른 설명 없이 JSON만 출력):
{
    "title": [<!-- 추출된 책 제목 -->],
    "author": [<!-- 추출된 책 저자 -->],
    "category": [<!-- 추출된 책 분류/장르(소설/에세이/기술) -->],
    "author_intro": [<!-- 저자 특성 언급/요구사항 -->],
    "book_intro": [<!-- 줄거리 관련 언급/요구사항 -->],
    "table_of_contents": [<!-- 세부적인 키워드 언급/요구사항 -->],
    "purpose": [<!-- 사용자의 독서 목적/이유 목록 (예: '재미', '학습', '시간 때우기', '기분') -->],
    "implicit info": [<!-- 추천해야 할 책에 대한 암시적 정보/특징/분위기 목록 (예: '밝은 분위기', '특정 상황에 어울리는 책', '최신 기술 동향') -->]
}
""",
    template_format="jinja2",  # 변수 치환을 위한 탬플릿
)

# 2. 선호도 통합 프롬프트
consolidate_pref_prompt = PromptTemplate(
    input_variables=["existing_preferences", "new_preferences"],
    template="""
기존에 수집된 사용자 선호도 정보와 새로 추출된 선호도 정보가 주어졌다. 두 정보를 지능적으로 통합하여 중복을 제거하고 관련 내용을 요약/결합하여 최종 선호도 목록을 생성해라.

[기존 선호도]
{{ existing_preferences }}

[새로운 선호도]
{{ new_preferences }}

[통합된 최종 선호도 목록]
(아래 목록 형태로만 출력, 각 항목은 문자열 리스트)
- 항목1: ["통합 내용1", "통합 내용2"]
- 항목2: ["통합 내용3"]
...
""",
    template_format="jinja2",
)

# 3. Decision Prompt
decision_prompt_template = PromptTemplate(
    template="""
[대화 맥락]
사용자 대화 내역:
{{ history }}
사용자의 최신 질문: "{{ query }}"
수집된 사용자 선호도:
{{ preferences }}

[역할 및 목표]
{{ role_instructions }}
현재 대화 상황, 질문, 수집된 선호도를 분석하여 아래 두 가지 행동 중 하나만 결정하고 필요한 정보를 생성하라.
- "추천": 사용자가 명시적으로 추천을 요청했거나, 사용자의 선호도 정보(카테고리, 저자, 목적, 책 줄거리, 사용자 수준, 분위기 등)를 반드시 3개 이상 수집했을 때만 추천에 들어가라.
- "추가 질문": 추천하기에 정보가 부족하거나 모호할 때, **아직 수집되지 않았거나 더욱 구체적인 선호도 정보**를 얻기 위한 질문 생성. (예: 어떤 장르를 선호하시나요? 특정 작가를 찾으시나요? 책을 읽는 목적이 무엇인가요?)

[출력 형식] (반드시 아래 형식만 정확히 따를 것)
행동: <추천 또는 추가 질문>
추가 질문: <"추가 질문" 행동일 경우 구체적인 질문 생성, "추천" 행동일 경우 빈 문자열>
""",
    input_variables=["history", "query", "preferences", "role_instructions"],
    template_format="jinja2",
)

# 4. Final Query Generation Prompt
final_query_generation_template = PromptTemplate(
    template="""
[대화 요약]
{{ history }}

[사용자 요청]
{{ query }}

[페르소나 정보]
{{ persona_info }}

[사용자 선호도 요약]
{{ preferences }}

위 정보를 전부 활용하여, 도서 검색에 가장 유용한 **핵심 키워드 중심의 최종 검색 쿼리**를 한 문장으로 작성하라.
오직 검색 쿼리 문장만 출력하라.
""",
    input_variables=["history", "query", "persona_info", "preferences"],
    template_format="jinja2",
)

# 5. Refine Prompt - 추후 더 업데이트를 통해 정제 기능 활성화
refine_prompt = PromptTemplate(
    input_variables=["query"],
    template="""주어진 검색 쿼리를 분석하여, 검색 엔진이나 다음 단계에서 사용하기 좋은 명확하고 간결한 단일 문장으로 정제해라. 불필요한 설명 없이 오직 정제된 쿼리 문장만 출력해라.

- "~~비슷한", "~ 같은", "~ 분위기의" 라고 사용자가 언급하면, 해당 책/저자의 **특징(예: 장르, 분위기, 핵심 소재, 작가 스타일)**을 반영하여 확장하라. 책 제목이나 저자 이름은 절대 직접 포함하지 마라.
- 예시 (입력: 해리포터 시리즈물같은 판타지 소설 알려줘) ->
1. 마법학교 배경의 청소년 판타지 소설 추천
2. 선과 악의 대결을 다룬 영국 판타지 시리즈
3. 성장 서사를 담은 인기 판타지 소설

[원본 검색 쿼리]
{{ query }}

[정제된 검색 쿼리]
""",
    template_format="jinja2",
)

# 6. Query Expansion Prompt
query_expansion_prompt = PromptTemplate(
    input_variables=["query"],
    template="""주어진 원본 검색 쿼리를 바탕으로, 관련성이 높으면서도 다양한 측면을 탐색할 수 있는 확장된 검색 쿼리 3개를 생성해라. 확장된 쿼리는 원본 쿼리의 핵심 의도를 반드시 유지해야 한다. 다른 설명이나 서론 없이, 오직 번호(1., 2., 3.)가 매겨진 확장 쿼리 목록만 한 줄에 하나씩 출력해라.

[원본 검색 쿼리]
{{ query }}

[확장된 검색 쿼리 목록]
""",
    template_format="jinja2",
)

# 7. Re_ranking Prompt
re_ranking_prompt = PromptTemplate(
    input_variables=["query", "documents"],
    template="""사용자의 검색 쿼리는 다음과 같습니다: "{{ query }}"
다음은 검색된 도서 목록입니다 (내용은 일부만 표시됨):
{% for doc in documents %}
{{ loop.index }}. 제목: {{ doc.metadata.get('title', '제목 없음') }}, 저자: {{ doc.metadata.get('author', '저자 없음') }}, 내용 일부: {{ doc.page_content | truncate(200) }}
{% endfor %}

위 검색 결과를 사용자의 검색 쿼리 "{{ query }}"와의 관련성, 그리고 문서 내용의 충실도를 종합적으로 고려하여 가장 적합한 순서대로 재배치하라.
가장 관련성이 높은 도서를 목록의 맨 위에 배치하고, 순위가 매겨진 **도서 제목과 저자**만으로 결과를 다음 형식으로 출력하라.

[출력 형식 예시]
1. 제목: <가장 관련성 높은 책 제목>, 저자: <저자 이름>
2. 제목: <두 번째 관련성 높은 책 제목>, 저자: <저자 이름>
...

[리랭킹된 도서 목록]
""",
    template_format="jinja2",
)

# 8. HyDE Generation Prompt
hyde_generation_prompt = PromptTemplate(
    input_variables=["query"],
    template="""다음 검색 쿼리에 완벽하게 부합하는 **이상적인 가상의 책**을 추천하고 이를 기반으로 **간결한 요약(2-3 문장)**을 생성해라. 이 요약은 해당 쿼리로 책을 찾는 사용자가 가장 만족할 만한 내용을 담고 있어야 한다. 오직 생성된 요약 텍스트만 출력하라.

[검색 쿼리]
{{ query }}

[가상의 책 요약]
""",
    template_format="jinja2",
)

# 9. Persona prompt
literature_role = "너는 감성적이고 문학적인 도서 추천 챗봇이다. 사용자의 감정과 취향을 깊이 이해하고 공감하는 말투로 문학적인 표현을 사용하여 책을 추천해라."
science_role = "너는 정확하고 논리적인 과학/기술 도서 추천 챗봇이다. 사용자의 지식 수준과 관심 분야를 파악하고, 최신 정보와 기술 동향을 반영하여 체계적으로 책을 추천해라."
general_role = "너는 친절하고 신뢰할 수 있는 범용 도서 추천 챗봇이다. 친근한 말투로 다양한 분야의 책에 대해 균형 잡힌 시각으로 정보를 제공하고, 사용자의 요구사항에 맞춰 명확하고 이해하기 쉽게 책을 추천해라."


# 10. hyde_keyword_prompt
hyde_keyword_prompt = PromptTemplate(
    input_variables=["hyde_summary"],
    template="""다음은 사용자의 질문에 이상적으로 부합하는 가상의 책 요약입니다:
"{{ hyde_summary }}"

이 요약 내용에서 **핵심 키워드 5개**를 추출하여 쉼표(,)로 구분하여 나열해라. 오직 키워드 목록만 출력하라.

[핵심 키워드 목록]
""",
    template_format="jinja2",
)

##### Async Invoke function

In [144]:
async def async_invoke(chain: LLMChain, vars_dict: dict, step_name: str) -> dict:
    """LLMChain을 비동기로 실행하고 로깅"""
    try:
        # 입력 변수 일부 로깅
        log_vars = {
            k: (v[:100] + "..." if isinstance(v, str) and len(v) > 100 else v)
            for k, v in vars_dict.items()
        }
        logger.debug(f"[{step_name}] Chain 호출 시작. 입력 변수 일부: {log_vars}")
        # chain.invoke를 별도 스레드에서 실행하여 비동기 처리
        result = await asyncio.to_thread(chain.invoke, vars_dict)
        result_text = result.get("text", "")
        # 결과 텍스트 일부 로깅
        log_result = (
            result_text[:200] + "..." if len(result_text) > 200 else result_text
        )
        logger.debug(f"[{step_name}] Chain 호출 완료. 결과 일부: {log_result}")
        return result
    except Exception as e:
        logger.error(f"[{step_name}] Chain 호출 중 예외 발생: {e}", exc_info=True)
        return {"text": ""}  # 오류 시 빈 결과 반환


async def async_invoke_llm(prompt: str, step_name: str) -> str:
    """LLM 직접 호출을 비동기로 실행하고 로깅"""
    try:
        # 프롬프트 일부 로깅
        log_prompt = prompt[:200] + "..." if len(prompt) > 200 else prompt
        logger.debug(f"[{step_name}] LLM 호출 시작. 프롬프트 일부: {log_prompt}")
        # llm.invoke를 별도 스레드에서 실행
        response = await asyncio.to_thread(llm_clova.invoke, prompt)

        # 응답 객체에서 텍스트 추출
        if hasattr(response, "content"):
            result_text = response.content.strip()
        elif isinstance(response, str):  # ChatClovaX가 문자열 직접 반환하는 경우
            result_text = response.strip()
        else:
            result_text = str(response).strip()

        # 응답 텍스트 일부 로깅
        log_result = (
            result_text[:200] + "..." if len(result_text) > 200 else result_text
        )
        logger.debug(f"[{step_name}] LLM 호출 완료. 응답 일부: {log_result}")
        return result_text
    except Exception as e:
        logger.error(f"[{step_name}] LLM 호출 중 예외 발생: {e}", exc_info=True)
        return ""  # 오류 시 빈 문자열

##### Base RAG Pipeline

In [ ]:
def _extract_metadata_field(doc: Document, field: str, default: str = "N/A") -> str:
    """
    Document의 다양한 metadata 구조에서 필드 값을 추출 (예: title, ISBN 등).
    """
    candidates = []

    # Case 1: 평평한 구조
    candidates.append(doc.metadata)

    # Case 2: 1단계 중첩
    if "metadata" in doc.metadata and isinstance(doc.metadata["metadata"], dict):
        candidates.append(doc.metadata["metadata"])

        # Case 3: 2단계 중첩
        inner = doc.metadata["metadata"]
        if "metadata" in inner and isinstance(inner["metadata"], dict):
            candidates.append(inner["metadata"])

    # Case 4: Elasticsearch 구조 (_source.metadata)
    if "_source" in doc.metadata and isinstance(doc.metadata["_source"], dict):
        source_meta = doc.metadata["_source"].get("metadata", {})
        if isinstance(source_meta, dict):
            candidates.append(source_meta)

    # 실제 값 추출
    for meta in candidates:
        if field in meta:
            return (
                meta[field].strip()
                if isinstance(meta[field], str)
                else str(meta[field])
            )

    return default

In [ ]:
# --- RAG 파이프라인 기본 클래스 ---
class BaseRAGPipeline:
    def __init__(
        self, config, llm, embeddings, vectorstore, es_store, retriever, documents
    ):
        self.config = config  # 페르소나 등 설정 정보
        self.llm = llm  # 답변 생성 LLM
        self.embeddings = embeddings  # 임베딩 모델
        self.vectorstore = vectorstore  # Milvus 벡터 저장소
        self.es_store = es_store  # Elasticsearch 저장소
        self.retriever = retriever  # 최종 리트리버 (ISBN 병합 포함)
        # self.qa_chain = qa_chain # RetrievalQA 체인 (직접 사용 대신 내부 로직 구현)
        self.documents = documents  # 전체 원본 문서 목록 (ISBN 병합 시 사용)

        # 대화 상태 관리
        self.user_history: List[str] = []  # 사용자 발화 기록
        self.llm_history: List[str] = []  # 챗봇 응답 기록
        self.user_preferences: Dict[str, List[str]] = (
            self._initialize_preferences()
        )  # 사용자 선호도
        self.preferences_text: str = "수집된 선호도 없음"  # 사용자 선호도 요약 텍스트
        self.preference_update_count: int = 0  # 선호도 업데이트 횟수
        self.last_recommendations: List[Document] = []  # 마지막 추천 결과 (병합된 문서)
        self.last_action: Optional[str] = None  # 마지막 챗봇 행동 (추천 or 추가 질문)

        # LLM 체인 초기화
        self.extract_pref_chain = LLMChain(llm=self.llm, prompt=extract_pref_prompt_v2)
        self.decision_chain = LLMChain(llm=self.llm, prompt=decision_prompt_template)
        self.final_query_generation_chain = LLMChain(
            llm=self.llm, prompt=final_query_generation_template
        )
        self.refine_chain = LLMChain(llm=self.llm, prompt=refine_prompt)
        self.query_expansion_chain = LLMChain(
            llm=self.llm, prompt=query_expansion_prompt
        )
        self.re_ranking_chain = LLMChain(llm=self.llm, prompt=re_ranking_prompt)
        self.hyde_generation_chain = LLMChain(
            llm=self.llm, prompt=hyde_generation_prompt
        )
        self.hyde_keyword_chain = LLMChain(llm=self.llm, prompt=hyde_keyword_prompt)

    def _initialize_preferences(self) -> Dict[str, List[str]]:
        """선호도 딕셔너리 초기화"""
        return {
            "title": [],
            "author": [],
            "category": [],
            "author_intro": [],
            "book_intro": [],
            "table_of_contents": [],
            "purpose": [],
            "implicit info": [],
        }

    def robust_parse_decision_response(
        self, response_text: str
    ) -> tuple[Optional[str], str]:
        """LLM의 행동 결정 응답을 파싱 (형식 오류에 강건하게)"""
        action = None
        additional_question = ""

        # "행동:" 패턴 찾기
        action_match = re.search(r'행동\s*[:：]\s*"?([^"\n]+)"?', response_text)
        if action_match:
            action = action_match.group(1).strip().lower()
            # "추천" 또는 "추가 질문" 외의 값은 None 처리 (혹은 기본값 설정)
            if action not in ["추천", "추가 질문"]:
                logger.warning(
                    f"알 수 없는 행동 값 파싱됨: '{action}'. '추가 질문'으로 처리."
                )
                action = "추가 질문"

        # "추가 질문:" 패턴 찾기
        follow_match = re.search(
            r'추가\s*질문\s*[:：]\s*"?(.+)"?', response_text, re.DOTALL
        )
        if follow_match:
            additional_question = follow_match.group(1).strip()

        # 행동은 찾았는데 추가 질문 내용이 없는 경우 기본 메시지 설정
        if action == "추가 질문" and not additional_question:
            additional_question = "어떤 점이 궁금하신가요? 또는 어떤 책을 찾으시나요?"
            logger.warning(
                f"행동은 '추가 질문'이나 질문 내용 없음. 기본 질문 사용: '{additional_question}'"
            )

        # 행동 자체를 못 찾은 경우 기본값 '추가 질문' 설정
        if not action:
            logger.warning(
                f"행동 결정 파싱 실패: '{response_text}'. 기본 '추가 질문'으로 처리."
            )
            action = "추가 질문"
            if not additional_question:  # 추가 질문 내용도 없으면 기본 메시지
                additional_question = "요청을 이해하기 어려웠습니다. 어떤 책을 찾으시는지 다시 말씀해주시겠어요?"

        logger.info(
            f"행동 결정 파싱 결과: 행동='{action}', 추가 질문='{additional_question[:50]}...'"
        )
        return action, additional_question

    async def update_preferences_from_input(self, user_input: str) -> None:
        """사용자 입력에서 선호도를 추출하고 내부 상태 업데이트"""
        logger.info(f"사용자 입력에서 선호도 추출 시작: '{user_input[:100]}...'")
        # 선호도 추출 LLMChain 호출
        extract_result = await async_invoke(
            self.extract_pref_chain, {"text": user_input}, "선호도 추출"
        )
        extracted_text = extract_result.get("text", "{}")

        extracted_prefs: Dict[str, List[str]] = {}
        try:
            # LLM 응답에서 JSON 부분만 찾기 (설명 등 제외)
            json_match = re.search(r"\{.*\}", extracted_text, re.DOTALL)
            if json_match:
                # 찾은 JSON 문자열 파싱
                extracted_prefs_raw = json.loads(json_match.group(0))
                # 정의된 키만 사용하고 값 형식(리스트) 통일
                defined_keys = self.user_preferences.keys()
                for key, value in extracted_prefs_raw.items():
                    if key in defined_keys:
                        vals_to_add = []
                        if isinstance(
                            value, list
                        ):  # 값이 리스트면, 빈 값 제외하고 문자열로 변환
                            vals_to_add = [
                                str(item).strip()
                                for item in value
                                if item and str(item).strip()
                            ]
                        elif (
                            isinstance(value, str) and value.strip()
                        ):  # 값이 문자열이면, 공백 아니면 리스트에 추가
                            vals_to_add = [value.strip()]
                        # 유효한 값이 있을 때만 extracted_prefs에 추가
                        if vals_to_add:
                            extracted_prefs[key] = vals_to_add
                    else:
                        logger.warning(
                            f"추출된 선호도 키 '{key}'가 정의된 형식에 없음. 무시."
                        )
            else:
                logger.warning(
                    f"선호도 추출 결과에서 JSON 객체를 찾을 수 없음: {extracted_text}"
                )

        except json.JSONDecodeError as e:
            logger.error(
                f"선호도 추출 결과 JSON 파싱 실패: {e}. 원본 텍스트: {extracted_text}"
            )
        except Exception as e:
            logger.error(f"선호도 추출/처리 중 예외 발생: {e}", exc_info=True)

        if not extracted_prefs:
            logger.info("새로 추출된 유효한 선호도 정보가 없음")
            return  # 업데이트할 내용 없으면 종료

        # 기존 선호도와 비교하여 변경 사항 적용
        updated_something = False
        for key, new_values in extracted_prefs.items():
            # 기존 값 집합 생성 (중복 제거 및 빠른 조회)
            existing_values_set = set(self.user_preferences.get(key, []))
            # 새로 추가될 값 필터링 (기존에 없는 값만)
            added_values = [v for v in new_values if v not in existing_values_set]

            if added_values:  # 새로 추가된 값이 있으면
                self.user_preferences[key].extend(added_values)  # 기존 리스트에 추가
                updated_something = True
                logger.info(f"선호도 업데이트됨 [{key}]: {self.user_preferences[key]}")

        if updated_something:
            self.preference_update_count += 1
            logger.info(
                f"선호도 업데이트 완료. 누적 업데이트 횟수: {self.preference_update_count}"
            )
            # 선호도 요약 텍스트 업데이트
            self._update_preferences_text()
        else:
            logger.info("기존 선호도에서 변경된 내용 없음.")

    def _update_preferences_text(self):
        """현재 user_preferences를 기반으로 요약 텍스트 업데이트"""
        pref_items = []
        # 사용자 친화적 키 이름 매핑
        display_key_map = {
            "title": "관련 제목",
            "author": "선호 저자",
            "category": "선호 장르/분류",
            "author_intro": "저자 관련 요구",
            "book_intro": "내용 관련 요구",
            "table_of_contents": "목차/키워드 요구",
            "purpose": "독서 목적",
            "implicit info": "기타 희망 사항/분위기",
        }
        # 값이 있는 선호도 항목만 문자열로 변환
        for key, values in self.user_preferences.items():
            if values:
                display_key = display_key_map.get(
                    key, key
                )  # 매핑된 이름 사용, 없으면 원래 키 사용
                # 값들을 쉼표로 연결하여 표시
                pref_items.append(f"- {display_key}: {', '.join(values)}")

        # 최종 요약 텍스트 생성
        self.preferences_text = (
            "\n".join(pref_items) if pref_items else "수집된 선호도 없음"
        )
        logger.debug(f"업데이트된 선호도 요약 텍스트:\n{self.preferences_text}")

    async def get_final_query(self, current_user_query: str) -> str:
        """대화 기록, 선호도 등을 종합하여 최종 검색 쿼리 생성 및 정제"""
        logger.info("최종 검색 쿼리 생성 시작")
        persona = self.config.get("persona", "General")  # 설정된 페르소나 가져오기
        # 페르소나별 중요 정보 예시 (쿼리 생성 프롬프트에 제공)
        persona_info_map = {
            "Literature": "감성, 현재 기분, 선호하는 문학 장르 및 작가 스타일",
            "Science": "독자의 지식 수준(초심자/전문가), 관심 기술 분야, 문제 해결 목표",
            "General": "선호 장르, 책을 찾는 이유, 원하는 분위기나 난이도",
        }
        persona_info = persona_info_map.get(persona, persona_info_map["General"])

        # 최종 쿼리 생성 LLMChain 호출
        final_query_vars = {
            "history": "\n".join(
                self.user_history[-5:] + self.llm_history[-5:]
            ),  # 최근 대화 5턴
            "query": current_user_query,
            "persona_info": persona_info,
            "preferences": self.preferences_text,  # 업데이트된 선호도 요약
        }
        result_gen = await async_invoke(
            self.final_query_generation_chain, final_query_vars, "선호도 종합 쿼리 생성"
        )
        generated_query = result_gen.get("text", "").strip()
        logger.info(f"LLM 생성 쿼리 (정제 전): '{generated_query}'")

        query_to_use = generated_query  # 기본값은 생성된 쿼리

        # 생성된 쿼리가 있으면 정제 시도
        if generated_query:
            refine_result = await async_invoke(
                self.refine_chain, {"query": generated_query}, "쿼리 정제"
            )
            refined_query = (
                refine_result.get("text", "").strip().strip('"')
            )  # 양쪽 따옴표 제거
            logger.info(f"정제된 쿼리: '{refined_query}'")

            # 정제된 쿼리가 유효한지 검사 (너무 짧거나, 특정 부정 키워드 포함 등)
            negative_keywords = [
                "없",
                "못",
                "않",
                "오류",
                "잘못",
                "알 수 없",
                "죄송",
                "필요",
            ]
            is_invalid_refinement = (
                not refined_query
                or len(refined_query) < 3  # 너무 짧음
                or any(
                    keyword in refined_query for keyword in negative_keywords
                )  # 부정 키워드
                or "{" in refined_query
                or "}" in refined_query  # JSON 형식 오류
                or refined_query.lower()
                == generated_query.lower()  # 정제 결과가 원본과 같음 (LLM이 정제 실패 시)
            )

            if is_invalid_refinement:
                logger.warning(
                    f"정제된 쿼리('{refined_query}')가 유효하지 않아 정제 전 쿼리('{generated_query}') 사용."
                )
                # query_to_use는 이미 generated_query 이므로 변경 필요 없음
            else:
                query_to_use = refined_query  # 유효하면 정제된 쿼리 사용
        else:
            # 쿼리 생성이 아예 실패한 경우 원본 사용자 쿼리 사용
            logger.warning("선호도 종합 쿼리 생성 실패. 원본 사용자 쿼리를 사용.")
            query_to_use = current_user_query

        # 최종 결정된 쿼리가 너무 짧으면 원본 사용자 쿼리 사용
        if not query_to_use or len(query_to_use) < 3:
            logger.warning(
                f"최종 결정된 쿼리('{query_to_use}')가 너무 짧거나 비어있어 원본 사용자 쿼리('{current_user_query}') 사용."
            )
            query_to_use = current_user_query

        logger.info(f"최종 결정된 검색 쿼리: '{query_to_use}'")
        return query_to_use

    async def _summarize_chunk_with_llm(self, text: str) -> str:
        """주어진 텍스트(주로 책 소개 등)를 LLM을 이용해 짧게 요약"""
        # 텍스트 길이 검사
        if not text or len(text.strip()) < MIN_INFO_LENGTH:
            return "요약할 정보가 충분하지 않습니다."

        # LLM 입력 길이 제한 고려 (예: 4000자)
        max_len = 4000
        truncated_text = text[:max_len].strip()
        if not truncated_text:
            return "요약할 정보가 없습니다."  # 공백만 있는 경우

        # 요약 프롬프트
        prompt = f"다음 책 정보를 2~3 문장으로 핵심 내용만 명확하게 요약해줘:\n\n{truncated_text}\n\n요약:"
        summary = await async_invoke_llm(prompt, "청크 요약")

        # LLM 요약 결과 검증 (너무 짧거나, 실패 메시지 포함 등)
        if (
            not summary
            or len(summary) < 10
            or "요약할 정보가" in summary
            or "죄송" in summary
            or "모르겠" in summary
        ):
            # LLM 요약 실패 시, 원본 텍스트 일부를 fallback으로 사용
            fallback_summary = text[:300].strip() + ("..." if len(text) > 300 else "")
            logger.warning(
                f"LLM 요약 실패 또는 부적절. Fallback 요약 사용: '{fallback_summary[:100]}...'"
            )
            return (
                fallback_summary
                if fallback_summary
                else "별도의 상세 정보가 충분치 않습니다."
            )

        return summary

    def _merge_documents_by_isbn(self, isbn: str) -> Optional[Document]:
        """주어진 ISBN에 해당하는 모든 원본 Document를 찾아 병합 (마스터 목록 사용)"""
        if not isbn:
            logger.warning("ISBN 없이 문서 병합 시도됨.")
            return None

        # self.documents (전체 원본 문서 목록)에서 해당 ISBN 찾기
        docs_for_isbn = [
            doc
            for doc in self.documents
            if str(doc.metadata.get("ISBN", "")).strip() == str(isbn).strip()
        ]

        if not docs_for_isbn:
            logger.warning(
                f"ISBN '{isbn}'에 해당하는 문서를 마스터 목록에서 찾을 수 없음."
            )
            return None  # 해당 ISBN 문서가 없으면 None 반환

        # 찾은 문서들의 page_content 병합
        combined_text = "\n\n---\n\n".join(
            doc.page_content for doc in docs_for_isbn if doc.page_content
        ).strip()

        if not combined_text:
            logger.warning(f"ISBN '{isbn}' 병합 후 텍스트 내용 없음.")
            # 텍스트 내용이 없어도 메타데이터는 유효할 수 있으므로 일단 반환 (호출부에서 처리)
            # 혹은 여기서 None을 반환할 수도 있음

        # 메타데이터는 첫 번째 문서의 것을 사용
        merged_meta = dict(docs_for_isbn[0].metadata)

        logger.debug(
            f"ISBN '{isbn}' 문서 병합 완료 (전체 원본 기준). 병합된 청크 수: {len(docs_for_isbn)}, 총 텍스트 길이: {len(combined_text)}"
        )
        return Document(page_content=combined_text, metadata=merged_meta)

    async def _embedding_rerank_documents(
        self, query: str, documents: List[Document]
    ) -> List[Document]:
        if not documents:
            logger.info("리랭킹할 문서가 없습니다.")
            return []

        # 쿼리 임베딩 준비
        embedding_tasks = {
            "main": asyncio.to_thread(self.embeddings.embed_query, query)
        }

        # 사용자 선호 임베딩 생성
        if self.user_preferences.get("author"):
            embedding_tasks["author"] = asyncio.to_thread(
                self.embeddings.embed_query, self.user_preferences["author"][0]
            )
        if self.user_preferences.get("title"):
            embedding_tasks["title"] = asyncio.to_thread(
                self.embeddings.embed_query, self.user_preferences["title"][0]
            )
        if self.user_preferences.get("category"):
            embedding_tasks["category"] = asyncio.to_thread(
                self.embeddings.embed_query, self.user_preferences["category"][0]
            )

        query_embeddings = await asyncio.gather(*embedding_tasks.values())
        query_embedding_map = dict(zip(embedding_tasks.keys(), query_embeddings))

        # 가중치 설정
        weights = {"main": 0.1, "author": 0.3, "title": 0.1, "category": 0.5}

        scored_docs = []

        for doc in documents:
            score_map: Dict[str, float] = {}

            # 실제 메타데이터 추출
            real_meta = doc.metadata.get("metadata", doc.metadata)

            # 1. 본문 유사도
            doc_embedding = await asyncio.to_thread(
                self.embeddings.embed_query, doc.page_content
            )
            score_map["main"] = cosine_similarity(
                [query_embedding_map["main"]], [doc_embedding]
            )[0][0]

            # 2. Author
            author_text = real_meta.get("author", "")
            if author_text and "author" in query_embedding_map:
                author_emb = await asyncio.to_thread(
                    self.embeddings.embed_query, author_text
                )
                score_map["author"] = cosine_similarity(
                    [query_embedding_map["author"]], [author_emb]
                )[0][0]

            # 3. Title
            title_text = real_meta.get("title", "")
            if title_text and "title" in query_embedding_map:
                title_emb = await asyncio.to_thread(
                    self.embeddings.embed_query, title_text
                )
                score_map["title"] = cosine_similarity(
                    [query_embedding_map["title"]], [title_emb]
                )[0][0]

            # 4. Category
            category_text = real_meta.get("category", "")
            if category_text and "category" in query_embedding_map:
                category_emb = await asyncio.to_thread(
                    self.embeddings.embed_query, category_text
                )
                score_map["category"] = cosine_similarity(
                    [query_embedding_map["category"]], [category_emb]
                )[0][0]

            # 정규화된 가중치 기반 점수 계산
            valid_keys = score_map.keys()
            total_weight = sum(weights[k] for k in valid_keys)
            final_score = sum(
                (weights[k] / total_weight) * score_map[k] for k in valid_keys
            )

            scored_docs.append((doc, final_score))

        # 점수 기준 정렬
        sorted_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
        logger.info("리랭킹 완료 (정규화 가중치 방식)")

        for i, (doc, score) in enumerate(sorted_docs):
            title = _extract_metadata_field(doc, "title", default="제목 없음")
            isbn = _extract_metadata_field(doc, "ISBN", default="N/A")
            logger.info(
                f"{i+1}. 제목: {title} | ISBN: {isbn} | 점수: {round(score, 4)}"
            )

        reranked_docs = [
            doc for doc, _ in sorted(scored_docs, key=lambda x: x[1], reverse=True)
        ]
        logger.info("리랭킹 완료 (정규화 가중치 방식, metadata 이중 구조 대응).")
        return reranked_docs

    async def _expand_query(self, query: str) -> List[str]:
        """LLM을 사용하여 원본 쿼리를 확장된 쿼리 여러 개로 생성"""
        logger.info(f"검색 쿼리 확장 시작: '{query}'")
        # 쿼리 확장 LLMChain 호출
        expansion_result = await async_invoke(
            self.query_expansion_chain, {"query": query}, "검색 쿼리 확장"
        )
        expanded_queries_text = expansion_result.get("text", "").strip()
        expanded_queries = []
        # LLM 응답 파싱 (번호 매겨진 리스트 형식 가정)
        for line in expanded_queries_text.splitlines():
            line_stripped = line.strip()
            if not line_stripped:
                continue  # 빈 줄 건너뛰기
            # "1. 확장쿼리 내용" 에서 쿼리 내용만 추출
            query_part = re.sub(r"^\d+\.\s*", "", line_stripped).strip()
            # 유효하고 원본과 다른 쿼리만 추가
            if query_part and query_part.lower() != query.lower():
                expanded_queries.append(query_part)

        # 최대 3개까지만 사용
        final_expanded_queries = expanded_queries[:3]
        logger.info(
            f"생성된 확장 쿼리 ({len(final_expanded_queries)}개): {final_expanded_queries}"
        )
        return final_expanded_queries

    async def _retrieve_documents(
        self, query: str, use_hyde: bool = False
    ) -> List[Document]:
        """
        최종 리트리버(Ensemble + ISBN Merging)를 사용하여 문서를 검색합니다.
        use_hyde가 True이고 조건 충족 시, HyDE 기법을 적용하여 검색 쿼리를 보강합니다.
        """
        retrieval_query = query
        hyde_summary = ""

        # HyDE 적용 조건: use_hyde=True 이고, 'implicit info' 선호도가 있을 때
        if use_hyde and self.user_preferences.get("implicit info"):
            logger.info("HyDE 활성화: 가상 문서 요약 및 키워드 추출 시도")
            # 1. 가상 문서 요약 생성
            hyde_result = await async_invoke(
                self.hyde_generation_chain, {"query": query}, "HyDE 가상 문서 생성"
            )
            hyde_summary = hyde_result.get("text", "").strip()

            if hyde_summary:
                logger.info(f"생성된 가상 문서 요약: '{hyde_summary[:200]}...'")
                # 2. 요약에서 키워드 추출
                keyword_result = await async_invoke(
                    self.hyde_keyword_chain,
                    {"hyde_summary": hyde_summary},
                    "HyDE 키워드 추출",
                )
                hyde_keywords_text = keyword_result.get("text", "").strip()
                # 쉼표로 구분된 키워드를 리스트로 변환
                hyde_keywords = [
                    k.strip() for k in hyde_keywords_text.split(",") if k.strip()
                ]

                if hyde_keywords:
                    logger.info(f"추출된 HyDE 키워드: {hyde_keywords}")
                    # 3. 원본 쿼리 + HyDE 키워드로 최종 검색 쿼리 구성
                    retrieval_query = f"{query} {' '.join(hyde_keywords)}"
                    logger.info(f"HyDE 적용된 검색 쿼리: '{retrieval_query}'")
                else:
                    logger.warning(
                        "HyDE 요약에서 키워드를 추출하지 못했습니다. 원본 쿼리 사용."
                    )
                    # retrieval_query는 원본 query 유지
            else:
                logger.warning("HyDE 가상 문서 요약 생성 실패. 원본 쿼리 사용.")
                # retrieval_query는 원본 query 유지
        elif use_hyde:
            # use_hyde는 True지만 implicit info가 없을 때
            logger.info("HyDE 조건 미충족 ('implicit info' 없음). 일반 쿼리 검색 수행.")
        else:
            # use_hyde 자체가 False일 때
            logger.info("일반 쿼리 검색 수행.")

        logger.info(f"최종 리트리버 호출 시작 (쿼리: '{retrieval_query[:100]}...')")
        try:
            # 최종 리트리버(merged_hybrid_retriever) 호출
            retrieved_docs = await self.retriever.aget_relevant_documents(
                retrieval_query
            )
            logger.info(f"검색된 문서 수 (ISBN 병합 완료됨): {len(retrieved_docs)}")
        except Exception as e:
            # 리트리버 실행 중 발생할 수 있는 모든 예외 처리
            logger.error(
                f"문서 검색 실패 (쿼리: '{retrieval_query}'): {e}", exc_info=True
            )
            retrieved_docs = []  # 오류 시 빈 목록 반환

        return retrieved_docs

    async def _generate_recommendations(self, final_query: str) -> str:
        """
        최종 검색 쿼리를 사용하여 문서를 검색, 확장, 정렬, 리랭킹하고
        상위 결과를 요약하여 최종 추천 응답 문자열을 생성합니다.
        """
        logger.info(f"추천 생성 시작. 최종 쿼리: '{final_query}'")

        # 1. 초기 검색 (HyDE 적용 가능성 있음)
        # 'implicit info' 선호도가 있으면 HyDE 시도
        use_hyde = False
        use_expansion = False

        # 명시적 author/category 없고, implicit info만 있을 때만 hyde + expansion
        has_author = bool(self.user_preferences.get("author"))
        has_title = bool(self.user_preferences.get("title"))
        has_implicit = bool(self.user_preferences.get("implicit info"))

        if not has_author and not has_title and has_implicit:
            use_hyde = True
            use_expansion = True

        # 2. 최초 검색 수행 (HyDE 적용 여부 반영)
        initial_docs = await self._retrieve_documents(final_query, use_hyde=use_hyde)

        # # 통합된 전체 문서 리스트

        # 그대로 사용 (이미 ISBN 병합된 상태)
        all_docs_to_consider = list(initial_docs)
        processed_queries = {final_query.lower()}

        # 2. 확장 쿼리 적용 (필요한 경우에만)
        if use_expansion:
            expanded_queries = await self._expand_query(final_query)
            expansion_tasks = []

            if expanded_queries:
                logger.info(
                    f"확장 쿼리 ({len(expanded_queries)}개)로 추가 검색 수행..."
                )
                for eq in expanded_queries:
                    eq_lower = eq.lower()
                    if eq_lower not in processed_queries:
                        expansion_tasks.append(
                            self._retrieve_documents(eq, use_hyde=False)
                        )
                        processed_queries.add(eq_lower)

                if expansion_tasks:
                    expansion_results = await asyncio.gather(*expansion_tasks)

                    # ISBN 기준 추가 (중복 방지)
                    existing_isbns = {
                        doc.metadata.get("ISBN")
                        for doc in all_docs_to_consider
                        if doc.metadata.get("ISBN")
                    }
                    for docs_list in expansion_results:
                        for doc in docs_list:
                            isbn = doc.metadata.get("ISBN")
                            if isbn and isbn not in existing_isbns:
                                all_docs_to_consider.append(doc)
                                existing_isbns.add(isbn)

        logger.info(
            f"초기 및 확장 검색 후 고려할 총 고유 문서 수: {len(all_docs_to_consider)}"
        )

        logger.debug(
            f"--- 리랭킹 함수(_embedding_rerank_documents) 입력 문서 메타데이터 확인 (총 {len(all_docs_to_consider)}개) ---"
        )
        for i, doc in enumerate(all_docs_to_consider):
            metadata_content = doc.metadata if doc.metadata else {}  # None 방지
            logger.debug(
                f"문서 {i+1} Metadata: {json.dumps(metadata_content, indent=2, ensure_ascii=False)}"
            )
            try:
                isbn_check = metadata_content.get("ISBN", "ISBN 키 없음")
                title_check = metadata_content.get("title", "Title 키 없음")
                logger.debug(f"  -> 확인: ISBN='{isbn_check}', Title='{title_check}'")
            except Exception as e:
                logger.error(f"  -> 메타데이터 접근 오류 발생: {e}")
        logger.debug("--- 리랭킹 입력 데이터 확인 완료 ---")

        # 검색 결과가 전혀 없으면 추천 불가 메시지 반환
        if not all_docs_to_consider:
            logger.warning("검색 및 확장 결과 문서를 찾지 못했습니다. 추천 생성 불가.")
            return "죄송합니다, 해당 조건에 맞는 책을 찾지 못했습니다. 다른 조건으로 다시 시도해 보시겠어요?"

        # 4. LLM 기반 리랭킹
        logger.info("리랭킹 수행...")
        ranked_docs = await self._embedding_rerank_documents(
            final_query, all_docs_to_consider
        )

        # 리랭킹 결과가 없으면 추천 불가
        if not ranked_docs:
            logger.warning("리랭킹 결과 유효한 문서가 없습니다. 추천 생성 불가.")
            return "죄송합니다, 추천할 책을 선정하는 데 어려움이 있습니다. 잠시 후 다시 시도해주세요."

        # 5. 상위 결과 선택 및 최종 추천 생성
        # 상위 3개 문서 선택
        top_docs = ranked_docs[:3]
        logger.info(f"최종 추천 후보 문서 수: {len(top_docs)}")

        recommendations = []  # 최종 응답에 포함될 추천 문구 리스트
        self.last_recommendations = []  # 후속 질문 처리를 위한 추천 문서 저장
        processed_isbns = set()  # 중복 추천 방지

        for rank, doc in enumerate(top_docs):
            isbn = _extract_metadata_field(doc, "ISBN")
            title = _extract_metadata_field(doc, "title", default="제목 없음")

            logger.debug(
                f"추천 후보 처리 중 (Rank {rank+1}): ISBN={isbn}, Title={title}"
            )

            if not isbn or isbn == "N/A":
                logger.warning(
                    f"Rank {rank+1} 추천 후보에 ISBN 없음. 건너뜀. Title: {title}"
                )
                continue
            if isbn in processed_isbns:
                logger.warning(f"중복 ISBN '{isbn}' 추천 목록에 이미 존재. 건너뜀.")
                continue

            # 후속 질문 처리를 위해, 해당 ISBN의 *전체* 내용이 담긴 문서를 master 목록에서 다시 가져와 저장
            # _re_rank_documents 에서 사용된 doc은 이미 ISBN 병합된 상태지만,
            # 명시적으로 전체 내용을 다시 로드하여 last_recommendations에 저장
            full_merged_doc = self._merge_documents_by_isbn(isbn)
            if not full_merged_doc:
                logger.error(
                    f"치명적 오류: 리랭킹된 문서(ISBN: {isbn})의 전체 정보를 병합하지 못했습니다. 추천 목록에서 제외."
                )
                continue
            # 병합된 전체 문서 내용을 last_recommendations에 저장
            self.last_recommendations.append(full_merged_doc)
            processed_isbns.add(isbn)  # 처리된 ISBN 기록

            # 사용자에게 보여줄 추천 문구 생성
            metadata = full_merged_doc.metadata
            title = metadata.get("title", "제목 정보 없음")
            author = metadata.get("author", "저자 정보 없음")
            book_cover = metadata.get("book_cover", "표지 정보 없음")

            # 요약할 텍스트 선택 (책소개 > 출판사리뷰 > 추천사 > 전체 내용 일부 순)
            book_intro = extract_field(full_merged_doc.page_content, "책소개")
            publisher_review = extract_field(full_merged_doc.page_content, "출판사리뷰")
            recommendation_field = extract_field(full_merged_doc.page_content, "추천사")

            text_for_summary = ""
            if book_intro and len(book_intro.strip()) >= MIN_INFO_LENGTH:
                text_for_summary = book_intro
            elif publisher_review and len(publisher_review.strip()) >= MIN_INFO_LENGTH:
                text_for_summary = publisher_review
            elif (
                recommendation_field
                and len(recommendation_field.strip()) >= MIN_INFO_LENGTH
            ):
                text_for_summary = recommendation_field
            else:
                # 필드 추출 실패 시, 전체 내용에서 앞부분 사용 (필드명 제외 시도)
                page_content_cleaned = re.sub(
                    r"^\s*.*?\s*[:：]\s*",
                    "",
                    full_merged_doc.page_content,
                    flags=re.MULTILINE,
                ).strip()
                text_for_summary = page_content_cleaned[:500]  # 길이 제한

            # LLM으로 요약 생성
            if text_for_summary and len(text_for_summary.strip()) >= MIN_INFO_LENGTH:
                summary = await self._summarize_chunk_with_llm(text_for_summary)
            else:
                summary = "책에 대한 상세 설명이 부족합니다."

            # 최종 추천 문구 형식
            # recommendation_text = (f"{rank+1}. \n표지: {book_cover}\n제목: {title}\n저자: {author}\n- 추천 이유: {summary}")
            recommendation_text = f"{rank+1}. \r\n표지: {book_cover}\r\n제목: {title}\r\n저자: {author}\r\n- 추천 이유: {summary}"
            recommendations.append(recommendation_text)
            logger.debug(f"추천 문구 생성됨: {title}")

        # 최종 추천 결과 로깅
        if self.last_recommendations:
            rec_titles = [
                d.metadata.get("title", "N/A") for d in self.last_recommendations
            ]
            logger.info(
                f"'_generate_recommendations' 종료. last_recommendations 업데이트 ({len(self.last_recommendations)}개): {rec_titles}"
            )
        else:
            logger.warning(
                "'_generate_recommendations' 종료. 최종 추천 목록(last_recommendations)이 비어있음!"
            )

        # 생성된 추천 문구가 없으면 실패 메시지 반환
        if not recommendations:
            return "추천할 만한 책을 찾지 못했습니다. 조건을 바꿔서 다시 질문해 주시겠어요?"

        # 최종 답변 조립
        final_answer = "이런 책들은 어떠신가요? 마음에 드는 책이 있다면 더 자세히 알려드릴게요.\n\n" + "\n\n".join(
            recommendations
        )
        logger.info("추천 응답 생성 완료.")
        return final_answer

    async def handle_followup_query(self, followup_query: str) -> tuple[bool, str]:
        """이전 추천에 대한 후속 질문인지 판단하고 처리"""
        logger.info(f"후속 질문 처리 시작. Query: '{followup_query}'")

        # 이전 추천 결과가 없으면 후속 질문 처리 불가
        if not self.last_recommendations:
            logger.error(
                "handle_followup_query 진입 오류: last_recommendations가 비어 있음!"
            )
            # 후속 질문이 아니라고 판단하고 False 반환
            return False, ""

        # LLM에게 후속 질문 의도 분석 요청
        # 이전 추천 목록 정보 제공
        rec_info = []
        for i, doc in enumerate(self.last_recommendations):
            title = doc.metadata.get("title", "제목 없음")
            isbn = doc.metadata.get("ISBN", "NO_ISBN")
            # 간단한 요약 정보만 제공 (LLM 컨텍스트 길이 고려)
            snippet = await self._summarize_chunk_with_llm(
                doc.page_content[:500]
            )  # 앞 500자만 요약
            rec_info.append(f"{i+1}. 제목: {title}, ISBN: {isbn}\n   요약: {snippet}")
        rec_info_str = "\n".join(rec_info)

        # 후속 질문 의도 분석 프롬프트
        prompt = f"""이전에 다음 책들을 추천했습니다:
{rec_info_str}

사용자의 후속 질문은 다음과 같습니다: "{followup_query}"

이 질문이 위 추천 목록과 관련된 후속 질문인지, 아니면 완전히 새로운 질문인지 판단하고, 후속 질문이라면 그 의도를 분석하여 다음 JSON 형식 중 **하나만** 출력해라. 새로운 질문이면 `{{ "action": "새 질문", "ISBN": null, "query": null }}` 형식으로 출력하라. **절대로 다른 설명이나 대화 없이 오직 JSON 객체 하나만 출력해야 한다.**

[후속 질문 의도 분류 및 JSON 형식]
- 특정 책 상세 정보 요청: `{{"action": "상세", "ISBN": "<요청된 책의 ISBN>", "query": "{followup_query}"}}` (몇 번째 책, 또는 제목으로 언급 시 해당 ISBN 사용)
- 특정 책과 유사한 책 추천 요청: `{{"action": "유사", "ISBN": "<기준 책의 ISBN>", "query": "<유사성 관련 사용자 언급>"}}`
- 추천된 책들 비교 요청: `{{"action": "비교", "ISBN": "<비교 대상 ISBN 목록 (쉼표 구분)>", "query": "{followup_query}"}}`
- 추천 결과에 대한 피드백/불만: `{{"action": "피드백", "ISBN": null, "query": "{followup_query}"}}` (예: "더 쉬운 책은 없나요?", "이건 별로네요")
- 완전히 새로운 질문: `{{"action": "새 질문", "ISBN": null, "query": null}}`

[주의사항]
- 사용자가 몇 번째 책이나 제목으로 언급하면, 제공된 추천 목록에서 정확한 ISBN을 찾아 사용하라. (예: 첫 번째 책이면 목록의 첫 ISBN)
- 여러 책을 언급하면 해당 ISBN들을 쉼표로 구분하여 포함하라.
- **출력은 반드시 유효한 JSON 객체 하나여야 한다. 다른 텍스트는 절대 포함하지 마라.**

[분석 결과 (JSON 객체만 출력)]
"""

        try:
            # 의도 분석 LLM 호출
            result_text = await async_invoke_llm(prompt, "후속 질문 의도 분석")
            logger.debug(f"후속 질문 의도 분석 LLM 원본 응답: {result_text}")

            # LLM 응답에서 JSON 파싱
            analysis_result = None
            json_match = re.search(r"\{.*\}", result_text, re.DOTALL)
            if json_match:
                try:
                    analysis_result = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    logger.error(
                        f"후속 질문 의도 분석 결과 JSON 파싱 실패 (JSON 형식 오류): {result_text}"
                    )
                    # 파싱 실패 시 새 질문으로 간주
                    analysis_result = {"action": "새 질문"}
            else:
                # JSON 객체를 찾지 못한 경우, 응답 텍스트 내용으로 추론 시도 (최후의 수단)
                if (
                    "새 질문" in result_text
                    or '"action": "새 질문"' in result_text.replace(" ", "")
                ):
                    logger.warning(
                        f"후속 질문 의도 분석 JSON 객체 미발견, '새 질문' 패턴 감지: {result_text}"
                    )
                    analysis_result = {"action": "새 질문"}
                else:
                    logger.error(
                        f"후속 질문 의도 분석 결과 JSON 파싱 실패 및 '새 질문' 패턴 미감지: {result_text}"
                    )
                    # 처리 불가로 판단, 후속 질문이 아닌 것으로 처리
                    return False, ""

            # 파싱된 결과에서 정보 추출
            action = analysis_result.get("action", "새 질문")  # 기본값 '새 질문'
            isbn_str = analysis_result.get("ISBN", "")
            query_part = analysis_result.get(
                "query", followup_query
            )  # query 필드 없으면 원본 사용

            logger.info(
                f"후속 질문 분석 결과: action='{action}', ISBN='{isbn_str}', query='{query_part[:50]}...'"
            )

            # 분석 결과에 따른 분기 처리
            if action == "새 질문":
                # 새 질문이면 처리하지 않고 False 반환
                return False, ""

            # ISBN 문자열을 리스트로 변환
            isbn_list = (
                [s.strip() for s in str(isbn_str).split(",") if s.strip()]
                if isbn_str
                else []
            )
            target_isbn = isbn_list[0] if isbn_list else None  # 주로 첫 번째 ISBN 사용

            # --- 상세 정보 요청 처리 ---
            if action == "상세":
                if not target_isbn:
                    # 대상 ISBN이 없으면 사용자에게 다시 질문
                    return (
                        True,
                        "어떤 책에 대해 더 알고 싶으신지 알려주시겠어요? (예: 첫 번째 책 또는 책 제목)",
                    )

                # last_recommendations에서 해당 ISBN의 전체 문서 찾기
                target_doc = next(
                    (
                        doc
                        for doc in self.last_recommendations
                        if str(doc.metadata.get("ISBN", "")).strip()
                        == str(target_isbn).strip()
                    ),
                    None,
                )

                if target_doc:
                    logger.debug(
                        f"상세 정보 요청: ISBN '{target_isbn}' 문서 찾음: Title='{target_doc.metadata.get('title')}'"
                    )
                    # 상세 설명 생성 프롬프트
                    detail_prompt = f"""다음은 사용자가 문의한 '{target_doc.metadata.get("title", "해당 책")}'에 대한 정보입니다. 이 정보를 바탕으로 사용자 질문 "{query_part}"에 답하거나, 특별한 질문이 없다면 책에 대해 자연스럽게 더 자세히 설명해주세요. 책의 특징, 주요 내용, 대상 독자 등을 포함하면 좋습니다. 관련 정보가 부족하면 솔직하게 말해주세요.

[책 정보 요약]
제목: {target_doc.metadata.get("title", "정보 없음")}
저자: {target_doc.metadata.get("author", "정보 없음")}
분류: {target_doc.metadata.get("category", "정보 없음")}
페이지: {target_doc.metadata.get("page", "정보 없음")} 쪽
가격: {target_doc.metadata.get("price", "정보 없음")} 원

[책 소개 및 내용 (일부)]
{target_doc.page_content[:3000]}...

[답변 또는 상세 설명]
"""
                    # 상세 설명 LLM 호출
                    detailed_info = await async_invoke_llm(
                        detail_prompt, "후속 상세 설명 생성"
                    )
                    return True, (
                        detailed_info
                        if detailed_info
                        else "죄송합니다, 요청하신 내용에 대한 추가 정보를 제공하기 어렵습니다."
                    )
                else:
                    # 해당 ISBN 문서를 못 찾은 경우
                    logger.warning(
                        f"상세 정보 요청 실패: target_isbn '{target_isbn}' 문서를 last_recommendations 에서 찾지 못함."
                    )
                    available_isbns = [
                        d.metadata.get("ISBN") for d in self.last_recommendations
                    ]
                    return (
                        True,
                        f"죄송합니다, 요청하신 ISBN({target_isbn})의 책 정보를 찾을 수 없습니다. 현재 추천된 책들의 ISBN은 다음과 같습니다: {available_isbns}",
                    )

            # --- 유사한 책 추천 요청 처리 ---
            elif action == "유사":
                if not target_isbn:
                    return (
                        True,
                        "어떤 책과 유사한 책을 찾으시는지 알려주시겠어요? (예: 두 번째 책 같은 스타일)",
                    )

                # 기준이 되는 책 정보 찾기
                base_doc = next(
                    (
                        doc
                        for doc in self.last_recommendations
                        if str(doc.metadata.get("ISBN", "")).strip()
                        == str(target_isbn).strip()
                    ),
                    None,
                )

                if base_doc:
                    logger.debug(
                        f"유사 책 요청: 기준 ISBN '{target_isbn}' 문서 찾음: Title='{base_doc.metadata.get('title')}'"
                    )
                    # 유사성 기반 새 검색 쿼리 생성
                    base_title = base_doc.metadata.get("title", "")
                    base_author = base_doc.metadata.get("author", "")
                    base_category = base_doc.metadata.get("category", "")
                    # 유사성 특징 추출 (LLM 사용 또는 간단한 규칙 기반)
                    # 예: "카테고리 장르의 저자 작가 스타일" 또는 "제목과 비슷한 분위기"
                    similarity_aspects = (
                        f"{base_category} 장르"
                        if base_category
                        else f"'{base_title}'와 비슷한"
                    )
                    if base_author:
                        similarity_aspects += f" {base_author} 작가 스타일"

                    # 사용자가 추가로 언급한 내용 반영 (query_part)
                    user_refinement = (
                        f" 그리고 '{query_part}' 특징을 가진"
                        if query_part and query_part != followup_query
                        else ""
                    )
                    # 새 검색 쿼리 조합
                    new_query = f"{similarity_aspects}{user_refinement} 책 추천"
                    logger.info(f"유사 책 추천을 위한 새 쿼리 생성: {new_query}")

                    # 생성된 새 쿼리로 다시 추천 생성 함수 호출
                    recommendation_result = await self._generate_recommendations(
                        new_query
                    )
                    return (
                        True,
                        f"네, '{base_title}'와(과) 비슷한 다른 책을 찾아볼게요.\n\n"
                        + recommendation_result,
                    )
                else:
                    # 기준 책을 못 찾은 경우
                    logger.warning(
                        f"유사 책 요청 실패: target_isbn '{target_isbn}' 기준 문서를 last_recommendations 에서 찾지 못함."
                    )
                    available_isbns = [
                        d.metadata.get("ISBN") for d in self.last_recommendations
                    ]
                    return (
                        True,
                        f"죄송합니다, 기준이 되는 ISBN({target_isbn})의 책 정보를 찾을 수 없습니다. 현재 추천된 책들의 ISBN은 다음과 같습니다: {available_isbns}",
                    )

            # --- 책 비교 요청 처리 ---
            elif action == "비교":
                if len(isbn_list) < 2:
                    return (
                        True,
                        "비교할 책을 두 권 이상 알려주시겠어요? (예: 첫 번째랑 세 번째 책 비교해주세요)",
                    )

                # 비교 대상 ISBN들이 실제로 last_recommendations에 있는지 확인
                valid_comparison_isbns = [
                    isbn
                    for isbn in isbn_list
                    if any(
                        str(d.metadata.get("ISBN", "")).strip() == str(isbn).strip()
                        for d in self.last_recommendations
                    )
                ]

                if len(valid_comparison_isbns) < 2:
                    logger.warning(
                        f"비교 요청 실패: 유효한 비교 대상 ISBN 부족 ({valid_comparison_isbns} / 요청: {isbn_list})"
                    )
                    available_isbns = [
                        d.metadata.get("ISBN") for d in self.last_recommendations
                    ]
                    return (
                        True,
                        f"죄송합니다, 비교 요청하신 책({isbn_list}) 중 일부를 찾을 수 없거나 유효하지 않습니다. 현재 추천된 책 ISBN: {available_isbns}",
                    )

                # 비교 처리 함수 호출
                comparison_result = await self._handle_comparison(
                    query_part, valid_comparison_isbns
                )
                return True, comparison_result

            # --- 피드백/불만 처리 ---
            elif action == "피드백":
                logger.info(f"사용자 피드백/불만 처리 시도: '{query_part}'")
                # 피드백을 반영하여 새 검색 쿼리 생성 (간단한 방식)
                feedback_based_query = f"이전 추천에 대해 '{query_part}' 라는 피드백이 있었습니다. 이 점을 고려하여 다른 책을 추천해주세요."
                logger.info(f"피드백 기반 새 쿼리 생성: {feedback_based_query}")

                # 새 쿼리로 다시 추천 생성 함수 호출
                recommendation_result = await self._generate_recommendations(
                    feedback_based_query
                )
                return (
                    True,
                    "피드백 감사합니다. 말씀해주신 점을 바탕으로 다른 책을 찾아보겠습니다.\n\n"
                    + recommendation_result,
                )

            # --- 처리되지 않은 action ---
            else:
                logger.warning(
                    f"처리되지 않은 후속 질문 action: {action}. 새 질문으로 간주."
                )
                return False, ""  # 새 질문으로 처리

        except Exception as e:
            # 후속 질문 처리 중 예외 발생
            logger.error(f"후속 질문 처리 중 예외 발생: {e}", exc_info=True)
            return True, "후속 질문 처리 중 오류가 발생했습니다. 다시 시도해 주세요."

    async def _handle_comparison(self, query_part: str, isbn_list: List[str]) -> str:
        """주어진 ISBN 목록의 책들을 비교 설명하는 기능"""
        logger.info(
            f"도서 비교 시작. ISBN 목록: {isbn_list}, 비교 관점: '{query_part}'"
        )
        comparison_docs_info = []  # 비교할 책들의 정보 저장 리스트

        # last_recommendations에서 비교 대상 책 정보 추출
        for isbn in isbn_list:
            doc = next(
                (
                    d
                    for d in self.last_recommendations
                    if str(d.metadata.get("ISBN", "")).strip() == str(isbn).strip()
                ),
                None,
            )
            if doc:
                metadata = doc.metadata
                # 비교에 사용할 정보 추출 (제목, 저자, 요약, 주요 메타데이터)
                summary = await self._summarize_chunk_with_llm(
                    doc.page_content[:2000]
                )  # 비교를 위해 조금 더 긴 내용 요약
                comparison_docs_info.append(
                    {
                        "title": metadata.get("title", "제목 없음"),
                        "author": metadata.get("author", "저자 없음"),
                        "summary": summary,
                        "category": metadata.get("category", "분류 없음"),
                        "page": metadata.get("page", "페이지 수 없음"),
                        "price": metadata.get("price", "가격 정보 없음"),
                        "isbn": isbn,  # 식별용 ISBN 추가
                    }
                )
            else:
                # 이 경우는 이론상 발생하면 안됨 (handle_followup_query에서 검증했으므로)
                logger.error(
                    f"비교 오류: 유효성 검사 후에도 ISBN '{isbn}' 문서를 last_recommendations에서 찾을 수 없음."
                )

        # 비교 대상이 2개 미만이면 비교 불가
        if len(comparison_docs_info) < 2:
            logger.warning(
                f"비교 가능한 문서가 2개 미만입니다 (찾은 문서 수: {len(comparison_docs_info)})."
            )
            return "비교할 책 정보를 충분히 찾지 못했습니다."

        # 비교 설명 생성 프롬프트
        comparison_prompt_template = """다음은 사용자가 비교를 요청한 책들의 정보입니다. 사용자의 비교 요청 관점인 "{{ query }}"에 특히 초점을 맞춰 이 책들을 명확하게 비교 설명해주세요. 각 책의 주요 특징, 장르, 내용 스타일, 난이도, 분량, 가격 등 관련 정보를 활용하고, 어떤 독자에게 더 적합할지 등을 포함하여 답변하는 것이 좋습니다. 표 형식보다는 자연스러운 문장으로 설명해주세요.

[비교 대상 책 정보]
{% for doc in summaries %}
--- 책 {{ loop.index }} (ISBN: {{ doc.isbn }}) ---
제목: {{ doc.title }}
저자: {{ doc.author }}
분류: {{ doc.category }}
페이지 수: {{ doc.page }}
가격: {{ doc.price }} 원
요약: {{ doc.summary }}
{% endfor %}

[사용자 비교 요청 관점/질문]
"{{ query }}"

[비교 설명]
"""
        try:
            # 프롬프트 템플릿 렌더링
            comp_template = PromptTemplate(
                template=comparison_prompt_template,
                input_variables=["summaries", "query"],
                template_format="jinja2",
            )
            rendered_prompt = comp_template.render(
                summaries=comparison_docs_info, query=query_part
            )
            logger.debug(
                f"도서 비교 프롬프트 생성 완료 (일부):\n{rendered_prompt[:500]}..."
            )

            # 비교 설명 생성 LLM 호출
            comparison_result = await async_invoke_llm(
                rendered_prompt, "도서 비교 설명 생성"
            )

            # 결과 검증
            if (
                not comparison_result
                or len(comparison_result) < 20
                or "죄송" in comparison_result
                or "모르겠" in comparison_result
            ):
                logger.warning("LLM 기반 도서 비교 설명 생성 실패 또는 결과 부적절.")
                return "죄송합니다, 요청하신 책들을 비교 설명하는 데 어려움이 있습니다."

            logger.info("도서 비교 설명 생성 완료.")
            return comparison_result

        except Exception as e:
            logger.error(f"도서 비교 설명 생성 중 예외 발생: {e}", exc_info=True)
            return "죄송합니다, 책들을 비교하는 중 오류가 발생했습니다."

    async def process_query(
        self, user_query: str, force_recommendation: bool = False
    ) -> str:
        """
        사용자 쿼리를 받아 처리하는 메인 로직.
        선호도 업데이트, 행동 결정(추가 질문 vs 추천), 후속 질문 처리 등을 수행.
        """
        logger.info(f"=== 새로운 사용자 쿼리 처리 시작: '{user_query}' ===")
        # 사용자 발화 기록 추가
        self.user_history.append(f"사용자: {user_query}")

        # 1. 사용자 입력에서 선호도 업데이트
        await self.update_preferences_from_input(user_query)

        # 2. 후속 질문 처리 시도 (이전 행동이 '추천'이었고, 추천 결과가 있을 경우)
        is_potential_followup = self.last_action == "추천" and self.last_recommendations
        if is_potential_followup:
            logger.info("이전 추천에 대한 후속 질문 가능성 확인 중...")
            handled, followup_output = await self.handle_followup_query(user_query)
            if handled:
                logger.info("후속 질문 처리 완료.")
                self.llm_history.append(f"챗봇: {followup_output}")  # 챗봇 응답 기록
                return followup_output  # 후속 질문 처리 결과 반환

            else:
                # 후속 질문이 아니라고 판단되면, 일반 질문 처리 로직으로 넘어감
                logger.info("후속 질문이 아님. 일반 질문 처리 로직으로 진행합니다.")

        # 3. 일반 질문 처리: 행동 결정 (추천 vs 추가 질문)
        action: Optional[str] = None
        additional_question: str = ""

        if not force_recommendation:  # 강제 추천 모드가 아닐 경우
            logger.info("행동 결정 요청 (추천 vs 추가 질문)")
            # 선호도 수집 상태 확인 (예: 제목 외 유의미한 선호도 1개 이상 & 업데이트 1회 이상)
            meaningful_prefs_count = sum(
                1 for k, v in self.user_preferences.items() if v and k != "title"
            )
            # 추천을 위한 최소 조건 설정 (예: 유의미 선호도 1개 이상 or 업데이트 2회 이상)
            should_recommend_heuristically = (
                meaningful_prefs_count >= 1 or self.preference_update_count >= 2
            )

            if not should_recommend_heuristically and self.preference_update_count < 2:
                # 초기 대화 단계에서는 추가 질문 유도
                logger.info(
                    f"선호도 부족 ({meaningful_prefs_count}개 / {self.preference_update_count}번 업데이트). '추가 질문' 강제 실행."
                )
                action = "추가 질문"
                additional_question = "어떤 종류의 책을 찾으시는지 좀 더 자세히 말씀해주시겠어요? (예: 구체적인 카테고리, 특정 작가, 책의 수준/분위기 등)"
            else:
                # 정보가 어느 정도 모였으면 LLM에게 판단 요청
                prompt_vars = {
                    "history": "\n".join(
                        self.user_history[-5:] + self.llm_history[-5:]
                    ),  # 최근 대화 5턴
                    "query": user_query,
                    "preferences": self.preferences_text,
                    "role_instructions": self.config.get(
                        "role_instructions", general_role
                    ),  # 설정된 페르소나 역할 지침
                }
                try:
                    decision_result = await async_invoke(
                        self.decision_chain, prompt_vars, "행동 결정"
                    )
                    decision_text = decision_result.get("text", "").strip()
                    action, additional_question_llm = (
                        self.robust_parse_decision_response(decision_text)
                    )
                    # LLM이 생성한 추가 질문 사용
                    if action == "추가 질문" and additional_question_llm:
                        additional_question = additional_question_llm

                except Exception as e:
                    # LLM 호출 실패 시 안전하게 '추가 질문'으로 처리
                    logger.error(
                        f"행동 결정 LLM 호출 실패: {e}. '추가 질문'으로 안전하게 진행.",
                        exc_info=True,
                    )
                    action = "추가 질문"
                    additional_question = "요청을 이해하는 데 어려움이 있었습니다. 어떤 책을 찾으시는지 다시 말씀해주시겠어요?"
        else:
            # 강제 추천 모드
            logger.info("강제 추천 모드 활성화. 행동='추천'")
            action = "추천"

        # 4. 결정된 행동 수행
        response = ""
        if action == "추가 질문":
            logger.info("행동: 추가 질문")
            self.last_action = "추가 질문"
            # 반복적인 추가 질문 방지 로직
            if additional_question:
                try:
                    # 추가 질문 임베딩 생성
                    add_q_emb = await asyncio.to_thread(
                        self.embeddings.embed_query, additional_question
                    )
                    # 이전 질문들과 유사도 비교
                    if is_similar_question(
                        add_q_emb,
                        previous_additional_question_embeddings,
                        threshold=0.90,
                    ):  # 임계값 높게 설정
                        logger.warning(
                            "이전과 매우 유사한 추가 질문 생성됨. 추천 강제 시도."
                        )
                        # 유사 질문 반복 시 강제로 추천 시도
                        return await self.process_query(
                            user_query, force_recommendation=True
                        )
                    else:
                        # 새로운 질문이면 임베딩 기록 추가 (최대 5개 유지)
                        previous_additional_question_embeddings.append(add_q_emb)
                        if len(previous_additional_question_embeddings) > 5:
                            previous_additional_question_embeddings.pop(0)
                        response = additional_question  # 생성된 추가 질문 사용
                except Exception as e:
                    logger.error(f"추가 질문 임베딩 또는 유사도 비교 중 오류: {e}")
                    response = additional_question  # 오류 시에도 일단 질문은 반환

            else:
                # 추가 질문 내용이 없는 이상한 경우 -> 추천 시도
                logger.warning("추가 질문 행동 결정되었으나 질문 내용 없음. 추천 시도.")
                action = "추천"

        if action == "추천":
            logger.info("행동: 추천")
            self.last_action = "추천"
            # 최종 검색 쿼리 생성
            final_query = await self.get_final_query(user_query)
            if not final_query:
                logger.error("최종 검색 쿼리 생성 실패. 추천 불가.")
                self.last_action = None  # 액션 상태 리셋
                response = "죄송합니다, 검색어를 만드는 데 실패했습니다. 다시 질문해 주시겠어요?"
            else:
                # 추천 생성 함수 호출
                recommendation_result = await self._generate_recommendations(
                    final_query
                )
                response = recommendation_result
                if (
                    not self.last_recommendations
                    and "죄송합니다" not in response
                    and "찾지 못했습니다" not in response
                ):
                    logger.warning(
                        "추천 생성 과정 완료 후 self.last_recommendations가 비어있으나, 응답은 성공 메시지 형태임."
                    )
                    # 이 경우 후속 질문 처리가 불가능할 수 있음
                    self.last_action = None  # 추천 실패로 간주하고 액션 리셋

        # 5. 최종 응답 반환 및 기록
        if response:
            self.llm_history.append(f"챗봇: {response}")  # 챗봇 응답 기록
            logger.info(
                f"챗봇 응답 생성 완료 (Action: {self.last_action}). 응답 일부: {response[:200]}..."
            )
            return response
        else:
            # 응답 생성 실패 시 (로직 오류 등)
            logger.error(f"최종 응답 생성 실패 (Action: {action}).")
            self.last_action = None  # 액션 상태 리셋
            fallback_msg = (
                "죄송합니다, 요청을 처리하는 중 문제가 발생했습니다. 다시 시도해주세요."
            )
            self.llm_history.append(f"챗봇: {fallback_msg}")
            return fallback_msg

    async def interactive_multi_turn_qa(self):
        """대화형 멀티턴 QA 세션 실행"""
        # 페르소나별 시작 인사말
        persona_greetings = {
            "Literature": "안녕하세요~ 감성적인 문학 도서 추천 챗봇입니다. 어떤 책을 찾으시나요? 당신의 이야기나 감정을 들려주셔도 좋아요.",
            "Science": "안녕하십니까. 과학/기술 도서 전문 챗봇입니다. 관심 분야, 알고 계신 내용, 찾으시는 정보의 깊이 등을 알려주시면 더 정확한 추천을 드릴 수 있습니다.",
            "General": "안녕하세요! 도서 추천 챗봇입니다. 어떤 종류의 책을 찾으시는지 편하게 말씀해주세요.",
        }
        persona = self.config.get("persona", "General")
        greeting = persona_greetings.get(persona, persona_greetings["General"])
        self.llm_history.append(f"챗봇: {greeting}")  # 대화 기록에 추가
        print("-" * 70)
        print(f"[{self.config.get('persona', '챗봇')}] {greeting}")  # 화면 출력
        print("-" * 70)

        turn_count = 0
        while True:
            turn_count += 1
            print(f"\n--- Turn {turn_count} ---")
            logger.info(f"--- Turn {turn_count} 시작 ---")
            # 디버깅 정보 로깅
            logger.debug(
                f"Turn 시작 | last_action: {self.last_action} | last_recs: {len(self.last_recommendations)}"
            )

            try:
                # 사용자 입력 받기
                user_query = input("[사용자] ").strip()
            except (KeyboardInterrupt, EOFError):
                print("\n[대화 종료]")
                break  # Ctrl+C 또는 EOF 시 종료

            # 종료 명령어 처리
            if user_query.lower() in ["quit", "exit", "종료", "그만"]:
                print("\n[대화 종료]")
                break

            # 빈 입력 무시
            if not user_query:
                continue

            # 사용자 쿼리 처리 및 응답 생성
            try:
                final_answer = await self.process_query(user_query)
            except Exception as e:
                # 처리 중 예외 발생 시 오류 메시지 출력 및 로깅
                logger.critical(
                    f"대화 처리 중 치명적 오류 발생 (Turn {turn_count}): {e}",
                    exc_info=True,
                )
                final_answer = "죄송합니다. 요청을 처리하는 중에 예상치 못한 오류가 발생했습니다. 잠시 후 다시 시도해주세요."
                # 오류 발생 시 상태 초기화 고려 (선택 사항)

            # 최종 응답 출력
            print("-" * 70)
            print(f"[{self.config.get('persona', '챗봇')}]\n{final_answer}")
            print("-" * 70)

##### 페르소나별 Pipeline 클래스

In [152]:
class LiteratureRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, embeddings, vectorstore, es_store, retriever, documents):
        config = {"persona": "Literature", "role_instructions": literature_role}
        super().__init__(
            config, llm, embeddings, vectorstore, es_store, retriever, documents
        )


class ScienceRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, embeddings, vectorstore, es_store, retriever, documents):
        config = {"persona": "Science", "role_instructions": science_role}
        super().__init__(
            config, llm, embeddings, vectorstore, es_store, retriever, documents
        )


class GeneralRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, embeddings, vectorstore, es_store, retriever, documents):
        config = {"persona": "General", "role_instructions": general_role}
        super().__init__(
            config, llm, embeddings, vectorstore, es_store, retriever, documents
        )

##### Main Execution

In [153]:
# --- 메인 실행 함수 ---
nest_asyncio.apply()  # Jupyter 환경 등에서 asyncio 이벤트 루프 중첩 허용


def main():
    # 이전 추가 질문 임베딩 초기화
    previous_additional_question_embeddings.clear()

    # 페르소나 선택
    print("페르소나를 선택해주세요:")
    print("1. 예술/문학 (감성적, 문학적 표현)")
    print("2. 과학/기술 (논리적, 정확한 정보)")
    print("3. 범용/일반 (친절, 균형잡힌 정보)")

    pipeline = None
    while pipeline is None:
        choice = input("원하는 페르소나 번호를 입력하세요 (1, 2, 3): ").strip()
        pipeline_map = {
            "1": LiteratureRAGPipeline,
            "2": ScienceRAGPipeline,
            "3": GeneralRAGPipeline,
        }
        if choice in pipeline_map:
            PipelineClass = pipeline_map[choice]
            try:
                # 선택된 페르소나 클래스로 파이프라인 인스턴스 생성
                pipeline = PipelineClass(
                    llm=llm_clova,
                    embeddings=ncp_embeddings,
                    vectorstore=vectorstore,  # Milvus
                    es_store=es_store,  # Elasticsearch
                    retriever=merged_hybrid_retriever,  # 최종 리트리버
                    documents=documents,  # 전체 원본 문서 목록
                )
                logger.info(f"선택된 페르소나: {pipeline.config['persona']}")
                print(f"\n'{pipeline.config['persona']}' 페르소나로 대화를 시작합니다.")
                print(
                    "대화를 종료하려면 'quit', 'exit', '종료', '그만' 중 하나를 입력하세요."
                )
            except Exception as e:
                # 파이프라인 초기화 실패 시
                logger.critical(
                    f"파이프라인 초기화 중 치명적 오류 발생: {e}", exc_info=True
                )
                print("오류: 파이프라인을 초기화할 수 없습니다. 로그를 확인하세요.")
                return  # 프로그램 종료
        else:
            print("잘못된 선택입니다. 1, 2, 3 중 하나를 입력해주세요.")

    # 대화형 세션 실행
    try:
        # 파이프라인의 interactive_multi_turn_qa 메소드를 비동기로 실행
        asyncio.run(pipeline.interactive_multi_turn_qa())
    except Exception as e:
        # 메인 실행 루프에서 예외 발생 시 로깅
        logger.critical(f"메인 실행 루프에서 치명적 오류 발생: {e}", exc_info=True)
    finally:
        # 프로그램 종료 전 Milvus 연결 해제 시도
        try:
            # 기본 Milvus 연결('default')이 있다면 해제
            if connections.get_connection_addr("default"):
                connections.disconnect("default")
                logger.info("Default Milvus 연결 해제 완료.")
        except Exception as e:
            # 연결 해제 실패는 경고로 처리
            logger.warning(f"Milvus 연결 해제 중 오류 발생 (무시 가능): {e}")
        logger.info("프로그램 종료.")


if __name__ == "__main__":
    main()

페르소나를 선택해주세요:
1. 예술/문학 (감성적, 문학적 표현)
2. 과학/기술 (논리적, 정확한 정보)
3. 범용/일반 (친절, 균형잡힌 정보)


2025-04-07 17:27:43,111 - INFO - 선택된 페르소나: Literature
2025-04-07 17:27:43,112 - INFO - --- Turn 1 시작 ---



'Literature' 페르소나로 대화를 시작합니다.
대화를 종료하려면 'quit', 'exit', '종료', '그만' 중 하나를 입력하세요.
----------------------------------------------------------------------
[Literature] 안녕하세요~ 감성적인 문학 도서 추천 챗봇입니다. 어떤 책을 찾으시나요? 당신의 이야기나 감정을 들려주셔도 좋아요.
----------------------------------------------------------------------

--- Turn 1 ---


2025-04-07 17:27:49,436 - INFO - === 새로운 사용자 쿼리 처리 시작: '히가시노 게이고의 추리소설 추천해줘.' ===
2025-04-07 17:27:49,437 - INFO - 사용자 입력에서 선호도 추출 시작: '히가시노 게이고의 추리소설 추천해줘....'
2025-04-07 17:27:51,201 - INFO - HTTP Request: POST https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-003 "HTTP/1.1 200 OK"
2025-04-07 17:27:51,204 - INFO - 선호도 업데이트됨 [author]: ['히가시노 게이고']
2025-04-07 17:27:51,205 - INFO - 선호도 업데이트됨 [category]: ['추리소설']
2025-04-07 17:27:51,205 - INFO - 선호도 업데이트됨 [implicit info]: ['유명 작가의 작품']
2025-04-07 17:27:51,206 - INFO - 선호도 업데이트 완료. 누적 업데이트 횟수: 1
2025-04-07 17:27:51,206 - INFO - 행동 결정 요청 (추천 vs 추가 질문)
2025-04-07 17:27:59,626 - INFO - HTTP Request: POST https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-003 "HTTP/1.1 200 OK"
2025-04-07 17:27:59,628 - INFO - 행동 결정 파싱 결과: 행동='추천', 추가 질문='...'
2025-04-07 17:27:59,629 - INFO - 행동: 추천
2025-04-07 17:27:59,629 - INFO - 최종 검색 쿼리 생성 시작
2025-04-07 17:28:00,447 - INFO - HTTP Request: POST https://clovastudio.stre

----------------------------------------------------------------------
[Literature]
이런 책들은 어떠신가요? 마음에 드는 책이 있다면 더 자세히 알려드릴게요.

1. 
표지: https://image.aladin.co.kr/product/25415/94/cover500/k702633114_1.jpg
제목: 추리소설가의 살인사건
저자: 히가시노 게이고 지음, 민경욱 옮김
- 추천 이유: 히가시노 게이고의 블랙 코미디 추리소설집으로, 추리 소설가, 편집자, 독자를 주제로 한 8개의 단편으로 구성되어 있다. 작가는 자신이 '추리소설가'라는 점을 활용하여 독특한 이야기를 선보이며, 빠른 호흡과 날카로운 유머로 독자를 사로잡는다. 특히, 첫 번째 수록작인 "세금 대책 살인사건"은 세금 문제를 다룬 경쾌한 추리물로, 많은 독자들에게 사랑받고 있다.

2. 
표지: https://image.aladin.co.kr/product/24265/94/cover500/8990982898_2.jpg
제목: 흑소 소설
저자: 히가시노 게이고 지음, 이혁재 옮김
- 추천 이유: 미스터리의 제왕 히가시노 게이고가 선보인 유머 소설 시리즈 중 하나로, 흑소라는 제목처럼 웃음 속에 인간과 사회의 비합리와 부조리를 향한 섬뜩한 풍자를 담고 있다. 특히 작가는 출판계와 문단의 뒷이야기를 다루며 일본 출판계의 상업 만능 풍조를 비판하며, 이외에도 일본 사회의 다양한 문제들을 풍자와 야유를 통해 꼬집는다. 또한 각 작품 속 인물들은 웃기지만 마냥 웃을 수만은 없는 우리 자신의 모습을 비추고 있어 깊은 여운을 남긴다.

3. 
표지: https://image.aladin.co.kr/product/26427/0/cover500/k182738525_1.jpg
제목: 그녀는 다 계획이 있다
저자: 히가시노 게이고 지음, 양윤옥 옮김
- 추천 이유: 컴패니언 교코가 일하는 하나야 보석점 고객 감사파티 후 동료 에리가 호텔 밀실에서 살해된다. 

2025-04-07 17:30:29,766 - INFO - Default Milvus 연결 해제 완료.
2025-04-07 17:30:29,766 - INFO - 프로그램 종료.



[대화 종료]
